# **1. Install and Import packages**

In [ ]:
#install bdpy for loading fMRI data
!pip install bdpy
!pip install SSIM-PIL

In [ ]:
#import packages
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from skimage import color
from skimage import io
import torch.nn as nn
import numpy as np
import pandas as pd
import imageio
import datetime
import io
import os
import h5py
import bdpy
import time
import torch
from PIL import Image
import imageio
import cv2
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch
import glob
import random
import os
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import csv
from sklearn.metrics import jaccard_score
import matplotlib.pyplot as plt # plotting library
import random
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
pip install opencv-python

In [ ]:
pip install opencv-python

# **2. Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **3. Load fMRI Data**

In [ ]:
#load Training fMRI data h5 using bdpy

full_training_data = bdpy.BData(r'C:\Users\Administrator\Desktop\Untitled Folder\data\fmri\sub-02_perceptionNaturalImageTraining_original_VC.h5')
#v1_train_fmri = full_training_data.select('ROI_V1')
#train_fmri = v1_train_fmri
stimulusID = full_training_data.select('stimulus_id')[0:6000]
train_hvc = full_training_data.select('ROI_LOC + ROI_FFA + ROI_PPA')[0:6000]
train_lvc = full_training_data.select('ROI_V1 + ROI_V2 + ROI_V3')[0:6000]
fmri_train_v1 = full_training_data.select('ROI_V1')[0:6000]
fmri_train_v2 = full_training_data.select('ROI_V2')[0:6000]
fmri_train_v3 = full_training_data.select('ROI_V3')[0:6000]
#full_training_data.show_metadata()
#stimulusID = full_training_data.select('stimulus_id')[0:1200]
#index = np.where(stimulusID == 3790512.014883)
#print(index)

In [ ]:
fmri_train_v1

In [ ]:
#load test fMRI dataset using bdpy

full_testing_data = bdpy.BData(r'C:\Users\Administrator\Desktop\Untitled Folder\data\fmri\sub-02_perceptionNaturalImageTest_original_VC.h5')
#v1_test_fmri = full_testing_data.select('ROI_V1')[0:50]
#test_fmri = v1_test_fmri
test_hvc = full_testing_data.select('ROI_LOC + ROI_FFA + ROI_PPA')[0:50]
test_lvc = full_testing_data.select('ROI_V1 + ROI_V2 + ROI_V3')[0:50]
fmri_test_v1 = full_testing_data.select('ROI_V1')[0:50]
fmri_test_v2 = full_testing_data.select('ROI_V2')[0:50]
fmri_test_v3 = full_testing_data.select('ROI_V3')[0:50]
stimulusID_test = full_testing_data.select('stimulus_id')[0:50]
#test_fmri =full_testing_data.get()[0:10]


# **Normalizing fMRI**

Train


In [ ]:
#Z score normalization
import pandas as pd
import numpy as np
import scipy.stats as stats
for i in range(6000):
  train_hvc[i] = stats.zscore(train_hvc[i])
  train_lvc[i] = stats.zscore(train_lvc[i])
  fmri_train_v1[i] = stats.zscore(fmri_train_v1[i])
  fmri_train_v2[i] = stats.zscore(fmri_train_v2[i])
  fmri_train_v3[i] = stats.zscore(fmri_train_v3[i])

In [ ]:
#normalization
for i in range(6000):
  train_hvc[i] = train_hvc[i]/train_hvc[i].max()
  train_lvc[i] = train_lvc[i]/train_lvc[i].max()
  fmri_train_v1[i] = fmri_train_v1[i]/fmri_train_v1[i].max()
  fmri_train_v2[i] = fmri_train_v2[i]/fmri_train_v2[i].max()
  fmri_train_v3[i] = fmri_train_v3[i]/fmri_train_v3[i].max()

Test

In [ ]:
#Z score normalization
import pandas as pd
import numpy as np
import scipy.stats as stats
for i in range(50):
  test_hvc[i] = stats.zscore(test_hvc[i])
  test_lvc[i] = stats.zscore(test_lvc[i])
  fmri_test_v1[i] = stats.zscore(fmri_test_v1[i])
  fmri_test_v2[i] = stats.zscore(fmri_test_v2[i])
  fmri_test_v3[i] = stats.zscore(fmri_test_v3[i])

In [ ]:
#normalization
for i in range(50):
  test_hvc[i] = test_hvc[i]/test_hvc[i].max()
  test_lvc[i] = test_lvc[i]/test_lvc[i].max()
  fmri_test_v1[i] = fmri_test_v1[i]/fmri_test_v1[i].max()
  fmri_test_v2[i] = fmri_test_v2[i]/fmri_test_v2[i].max()
  fmri_test_v3[i] = fmri_test_v3[i]/fmri_test_v3[i].max()

# **4. Train SM Decoder**

In [ ]:
#train_sm_decoder
import numpy as np
import argparse
import os
from sklearn import preprocessing
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import torchvision
import matplotlib.pyplot as plt
n_epochs = 200 #100 bood
batch_size = 50
lr =  0.002# 0.0002
b1 = 0.9
b2 = 0.999

np.random.seed(0)

#%cd /content/drive/My\ Drive/thesis/data/fMRI_data/demo1/
#full_training_data = bdpy.BData('sub-01_perceptionNaturalImageTraining_original_VC.h5')
#full_testing_data = bdpy.BData('sub-01_perceptionNaturalImageTest_original_VC.h5')
train_fmri = train_hvc
test_fmri = test_hvc
fmri = np.concatenate([train_fmri, test_fmri])
#fmri = torch.load('../data/fMRI_data/demo1/digits-fmri')

#lables of training data
# %cd /content
train_labels = []

# "convert tsv file into dictioary"
tsv_file = open(r"C:\Users\Administrator\Desktop\Untitled Folder\data\tsv\stimulus_NaturalImageTraining.tsv")
read_tsv = csv. reader(tsv_file, delimiter="\t")
df = pd.DataFrame(read_tsv)
df = df.set_axis(['Name', 'ID','label','num'], axis=1, inplace=False)

for i in range(0,6000):
  m = df['label'][df['ID'] == str(stimulusID[i][0])]
  train_labels.append(int(m)-1)


#lables of testing data
# %cd /content
test_labels = []

# "convert tsv file into dictioary" TEST
tsv_file = open(r"C:\Users\Administrator\Desktop\Untitled Folder\data\tsv\stimulus_NaturalImageTest.tsv")
read_tsv = csv. reader(tsv_file, delimiter="\t")
df = pd.DataFrame(read_tsv)
df = df.set_axis(['Name', 'ID','label','num'], axis=1, inplace=False)

for i in range(0,50):
  m = df['label'][df['ID'] == str(stimulusID_test[i][0])]
  e = int(m)-1
  test_labels.append(e+150)

In [ ]:
raw_labels = np.concatenate([train_labels, test_labels])
raw_labels = raw_labels.astype(float)
labels = raw_labels
newlabels = labels

total_blocks = fmri.shape[0]
fmri_size = fmri.shape[1]
print('total blocks:'+str(total_blocks))
print('input fmri size:'+str(fmri_size))

test_num = 50 # ghablan 10 bood man avaz kardam

cuda = True if torch.cuda.is_available() else False


class Decoder(nn.Module):
    def __init__(self, fmri_size):
        super(Decoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(fmri_size, 1024),
            nn.Tanh(),
            # nn.Linear(1024,512),
            # # nn.Tanh(),
            # nn.Linear(1024,512),
            # nn.Tanh(),
            nn.Linear(1024, 64),
            nn.Tanh()
        )

        self.classify = nn.Sequential(
            nn.Linear(64, 150), #ghablan 2 bood
            nn.Sigmoid()
        )

    def forward(self, fmri):
        fmri_flat = fmri.view(fmri.shape[0], -1)
        semantics = self.model(fmri_flat)
        x = self.classify(semantics)
        return x, semantics

# Use binary cross-entropy loss
adversarial_loss = torch.nn.CrossEntropyLoss()
encoder = Decoder(fmri_size=fmri_size)

if cuda:
    encoder.cuda()
    adversarial_loss.cuda()

# Optimizers
optimizer_E = torch.optim.Adam(encoder.parameters(), lr=lr, betas=(b1, b2))
scheduler_E = torch.optim.lr_scheduler.StepLR(optimizer_E, step_size=10, gamma=0.8, last_epoch=-1)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# ----------
#  Training
# ----------


fmri = np.concatenate([train_fmri, test_fmri])

train_labels = labels[0:6000]
test_labels = labels[6000:6050]
labels = np.concatenate([train_labels, test_labels])
labels = labels.astype(float)

fmri = torch.from_numpy(fmri)
fmri = fmri.type(Tensor)

train_fmri = torch.from_numpy(train_fmri)
train_labels = torch.from_numpy(train_labels)
train_labels = train_labels.squeeze()
train_fmri = train_fmri.type(Tensor)
train_labels = train_labels.type(Tensor)

test_fmri = torch.from_numpy(test_fmri)
test_labels = torch.from_numpy(test_labels)
test_labels = test_labels.squeeze()
test_fmri = test_fmri.type(Tensor)
test_labels = test_labels.type(Tensor)

acc_vec = []
train_vec = []
loss_vec = []
final_t_loss = []
for epoch in range(n_epochs):
    for i in range(0, 120):
            fmri_data = train_fmri[i*batch_size:(i+1) * batch_size]
            labels_data = train_labels[i*batch_size:(i+1) * batch_size]
            labels_data = labels_data.long()
            optimizer_E.zero_grad()
            predict, _ = encoder(fmri_data)
            print("shape of predicted")
            print(predict.shape)
            print("shape of labels")
            print(labels_data.shape)
            e_loss = adversarial_loss(predict, labels_data)
            e_loss.backward()
            optimizer_E.step()
            loss_vec.append(e_loss.item())
            print(
                "[Epoch %d/%d] [Batch %d] [loss: %f] "
                % (epoch, n_epochs, i, e_loss.item())
            )

    # if epoch % 1 == 0:
    #         test_fmri_data = test_fmri
    #         # test_label_data = test_labels.cpu().detach().numpy()
    #         test_labels = test_labels.long()
    #         lbs, _ = encoder(test_fmri_data)
    #         test_loss = adversarial_loss(lbs,test_labels)
    #         final_t_loss.append(test_loss.item())
            # cpu_labels = lbs.cpu().detach().numpy()
            # pred = [np.argmax(lb) for lb in cpu_labels]
            # num_correct = (pred == test_label_data).sum()
            # acc = num_correct / 50
            # acc_vec.append(acc)


    scheduler_E.step()
    lrd = optimizer_E.param_groups[0]['lr']


plt.figure()
plt.title('Loss of classification on training Data')
plt.plot(loss_vec)
plt.xlabel('Epoch num * number of batch')
plt.ylabel('Loss')
plt.show()

plt.figure()
plt.title('Loss of classification on test Data - 200 class')
plt.plot(final_t_loss)
plt.xlabel('Epoch num * number of batch')
plt.ylabel('Loss')
plt.show()

pred, sm = encoder(fmri)
torch.save(sm,'C:\\Users\\Administrator\\Desktop\\Untitled Folder\\data\\tsv\\semantics')
print(len(sm))

In [ ]:
np.savetxt("loss_semantic_train.csv", loss_vec, delimiter=",")

# **Augmentation Semantic Decoder**

In [ ]:
aug_label = []
for i in range(149):
  if i <=54:
    for j in range(8):
      aug_label.append(i)
  else:
    for j in range(8):
      aug_label.append(i+1)

for j in range(8):
  aug_label.append(55)

In [ ]:
#select Training images that correspond to stimulusID
import shutil
%cd /content/
original_path = '/content/drive/MyDrive/augmented_images/final_aug/'
target_path = '/content/drive/MyDrive/thesis/data/images/demo1/raw_imgs/'

new_name = 6000
for picture_name in range(7000,8200):

  new_original = os.path.join(original_path, str(picture_name) + ".jpg")
  if os.path.isfile(new_original):
    new_target = os.path.join(target_path, str(new_name) + ".JPEG")
    shutil.copyfile(new_original, new_target)
    #new_dir = os.path.join(target_path, str(new_name) + ".JPEG")
    #os.rename(file_path, new_dir)
  new_original = ""
  new_target = ""
  new_name = int(new_name) + 1

In [ ]:
#aug images


newSize = 128
patch_num = 128
aug_imgs = np.zeros((1200,newSize,newSize))
patch_aug_imgs = np.zeros((1200,patch_num,patch_num))
img_path = '/content/drive/MyDrive/thesis/data/images/demo1/raw_imgs/'
filelist = os.listdir(img_path)
num = 0


for d in range(6000,7200):
    new_aug = cv2.imread(img_path +'%d.JPEG' %d)
    new_imgs = color.rgb2gray(new_aug)
    new_imgs = cv2.resize(new_imgs, dsize = (newSize, newSize), interpolation=cv2.INTER_CUBIC)
    # image = patch_train_imgs[num,:,:]
    patch_aug_imgs[num,:,:] = cv2.normalize(new_imgs, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
    one = patch_aug_imgs[num,:,:]
    for i in range (128):
      for j in range(128):
          if one[i][j]<=0.5:
            one[i][j] = 1

          else:
            one[i][j] = 0

    patch_aug_imgs[num,:,:] = one
    num = num +1

In [ ]:
train_labels_new = train_labels.cpu().numpy()

In [ ]:
semantic_vecs = torch.load('/content/drive/MyDrive/thesis/data/sm_features/demo1/semantics')

In [ ]:
aug_semantic = np.zeros([1200,64])
import torch
num = 0
b = semantic_vecs.to(torch.device("cpu"))
c = b.detach().numpy()

for i in aug_label:

  address = np.where(train_labels_new == i)[0]
  d = c[address].mean(axis = 0)
  aug_semantic[num,:] = d
  num = num +1

In [ ]:
aug_train = np.concatenate((c[0:6000],aug_semantic), axis = 0)

In [ ]:
print(len(aug_train))

# **5. Change Name of Training Images**

In [ ]:
#select Training images that correspond to stimulusID
import shutil
%cd /content/
# "convert tsv file into dictioary"
tsv_file = open("/content/drive/MyDrive/Stimulus_excel/stimulus_NaturalImageTraining.tsv")
read_tsv = csv. reader(tsv_file, delimiter="\t")
df = pd.DataFrame(read_tsv)
df = df.set_axis(['Name', 'ID','label','num'], axis=1, inplace=False)

df['label'] = df['label'].astype(float, errors = 'raise')
train_labels = df["label"]
#train_labels = np.array(train_labels)
print(train_labels.dtype)
df.drop(['label','num'] , inplace=True, axis=1)

dict_train = dict(zip(df.ID, df.Name))
print(dict_train)
nameStimulus = []
# "find ID's that correspond to Image name"
for ID in stimulusID:
  #res = {sub['name'] : sub['age'] for ID in test_list}
  res = {float(key) : dict_train[key] for key in dict_train}
  convK = float(ID)
  res = res[convK]
  nameStimulus.append(res)


print(nameStimulus)
#directory_path = '/content/drive/MyDrive/thesis/data/images/demo1/raw_imgs/'
original_path = '/content/drive/MyDrive/thesis/data/images/demo1/result/training/'
# original_path = '/content/drive/MyDrive/thesis/data/images/demo1/fff/'
target_path = '/content/drive/MyDrive/thesis/data/images/demo1/raw_imgs/'

nameStimulus = nameStimulus[0:6000]
new_name = 0
for picture_name in nameStimulus:

  new_original = os.path.join(original_path, picture_name + ".png")
  if os.path.isfile(new_original):
    new_target = os.path.join(target_path, str(new_name) + ".JPEG")
    shutil.copyfile(new_original, new_target)
    #new_dir = os.path.join(target_path, str(new_name) + ".JPEG")
    #os.rename(file_path, new_dir)
  new_original = ""
  new_target = ""
  new_name = int(new_name) + 1

In [ ]:
#select Test images that correspond to stimulusID
import shutil
%cd /content/
# "convert tsv file into dictioary"
test_file = open("/content/drive/MyDrive/Stimulus_excel/stimulus_NaturalImageTest.tsv")
test_tsv = csv. reader(test_file, delimiter="\t")
df_test = pd.DataFrame(test_tsv)
df_test = df_test.set_axis(['Name', 'ID','label','num'], axis=1, inplace=False)

df_test['label'] = df_test['label'].astype(float, errors = 'raise')
test_labels = df_test['label']
df_test.drop(['label','num'] , inplace=True, axis=1)


dict_test = dict(zip(df_test.ID, df_test.Name))
#print(dict_test)

nameStimulus = []
# "find ID's that correspond to Image name"
for ID in stimulusID_test:

  res = {float(key) : dict_test[key] for key in dict_test}
  convK = float(ID)
  res = res[convK]
  nameStimulus.append(res)

original_path = '/content/drive/MyDrive/thesis/data/images/demo1/result/test/'
# original_path = '/content/drive/MyDrive/thesis/data/images/demo1/original_imgs/'
# original_path = '/content/drive/MyDrive/thesis/data/images/demo1/fff/'
target_path = '/content/drive/MyDrive/thesis/data/images/demo1/raw_imgs/'

new_name = 6000
nameStimulus = nameStimulus[0:50]
for picture_name in nameStimulus:

  new_original = os.path.join(original_path, picture_name + ".png")
  if os.path.isfile(new_original):
    new_target = os.path.join(target_path, str(new_name) + ".JPEG")
    shutil.copyfile(new_original, new_target)
    #new_dir = os.path.join(target_path, str(new_name) + ".JPEG")
    #os.rename(file_path, new_dir)
    new_original = ""
    new_target = ""
  else:
    new_original = ""
    new_target = ""
  new_name = int(new_name) + 1

# **6. Load Training and Test Images**

In [ ]:
#training images
import skimage.measure
import cv2
from keras.preprocessing import image

newSize = 128
patch_num = 128
train_imgs = np.zeros((6000,newSize,newSize))
patch_train_imgs = np.zeros((6000,patch_num,patch_num))
img_path = '/content/drive/MyDrive/thesis/data/images/demo1/raw_imgs/'
filelist = os.listdir(img_path)
num = 0


for d in range(0,6000):
    new_train = cv2.imread(img_path +'%d.JPEG' %d)
    new_imgs = color.rgb2gray(new_train)
    new_imgs = cv2.resize(new_imgs, dsize = (newSize, newSize), interpolation=cv2.INTER_CUBIC)
    # image = patch_train_imgs[num,:,:]
    patch_train_imgs[num,:,:] = cv2.normalize(new_imgs, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
    one = patch_train_imgs[num,:,:]
    for i in range (128):
      for j in range(128):
          if one[i][j]<=0.5:
            one[i][j] = 1

          else:
            one[i][j] = 0

    patch_train_imgs[num,:,:] = one
    num = num +1

In [ ]:
#test images
newSize = 128
patch_num = 128
test_imgs = np.zeros((50,newSize,newSize))
img_path = '/content/drive/MyDrive/thesis/data/images/demo1/raw_imgs/'
filelist = os.listdir(img_path)
patch_test_imgs = np.zeros((50,patch_num,patch_num))
num = 0

for d in range(6000,6050):

    new_test = cv2.imread(img_path +'%d.JPEG' %d)
    new_imgs = color.rgb2gray(new_test)
    new_imgs = cv2.resize(new_imgs, dsize = (newSize, newSize), interpolation=cv2.INTER_CUBIC)
    patch_test_imgs[num,:,:] = cv2.normalize(new_imgs, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
    one = patch_test_imgs[num,:,:]
    for i in range (128):
      for j in range(128):
          if one[i][j]<=0.5:
            one[i][j] = 1

          else:
            one[i][j] = 0

    patch_test_imgs[num,:,:] = one
    num = num + 1

In [ ]:
for i in range(50):
  plt.figure()
  plt.imshow(patch_test_imgs[i], cmap = 'gray')

In [ ]:
one_imgs = cv2.resize(patch_train_imgs[1,:,:], dsize = (32, newSize), interpolation=cv2.INTER_CUBIC)

In [ ]:
#convert Train Image to vec
obj_vector_train = np.zeros((6000,16384))

for i in range(6000):
  images = patch_train_imgs[i]

  flat_imgs = images.flatten()
  obj_vector_train[i] = flat_imgs

obj_vector_train = obj_vector_train.astype('float32')

In [ ]:
#convert Test Image to vec
obj_vector_test = np.zeros((50,16384))

for i in range(50):
  images = patch_test_imgs[i]

  flat_imgs = images.flatten()
  # for j in flat_imgs:
  #   j = float(j)
  obj_vector_test[i] = flat_imgs

obj_vector_test = obj_vector_test.astype('float32')

# **7. Functions**

In [ ]:
#plot function

def plot_train(loss_train,validation_loss,name,x_label,y_label):
  # plt.figure()
  # plt.title(name)
  # plt.plot(loss_train)
  # plt.plot(validation_loss)
  # plt.xlabel(x_label)
  # plt.ylabel(y_label)
  # plt.show()

  fig, (ax1, ax2) = plt.subplots(2)
  fig.suptitle('Vertically stacked subplots')
  ax1.plot(loss_train)
  ax2.plot(validation_loss)

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


##############################
#           U-NET
##############################


class UNetDown(nn.Module):
    def __init__(self, in_size, out_size= 3, normalize=True, dropout=0.0):
        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 4, 2, 1, bias=False)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_size))
        layers.append(nn.LeakyReLU(0.2))
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


class UNetUp(nn.Module):
    def __init__(self, in_size, out_size = 3, dropout=0.0):
        super(UNetUp, self).__init__()
        layers = [
            nn.ConvTranspose2d(in_size, out_size, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(out_size),
            nn.ReLU(inplace=True),
        ]
        if dropout:
            layers.append(nn.Dropout(dropout))

        self.model = nn.Sequential(*layers)

    def forward(self, x, skip_input):
        x = self.model(x)
        x = torch.cat((x, skip_input), 1)

        return x


##############################
#        Discriminator
##############################


class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalization=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(in_channels * 2, 64, normalization=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 3, 4, padding=1, bias=False)
        )

    def forward(self, img_A, img_B):
        # Concatenate image and condition image by channels to produce input
        img_input = torch.cat((img_A, img_B), 1)
        return self.model(img_input)


class GeneratorUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(GeneratorUNet, self).__init__()

        self.down1 = UNetDown(in_channels, 64, normalize=True)
        self.down2 = UNetDown(64, 128)
        self.down3 = UNetDown(128, 256)
        self.down4 = UNetDown(256, 512)
        self.down5 = UNetDown(512, 512)
        self.down6 = UNetDown(512, 512)
        self.down7 = UNetDown(512, 512)
        self.down8 = UNetDown(512, 512, normalize=False, dropout=0.5)

        self.up1 = UNetUp(576, 512, dropout=0.5)
        self.up2 = UNetUp(1024, 512, dropout=0.5)
        self.up3 = UNetUp(1024, 512, dropout=0.5)
        self.up4 = UNetUp(1024, 512, dropout=0.5)
        self.up5 = UNetUp(1024, 256)
        self.up6 = UNetUp(512, 128)
        self.up7 = UNetUp(256, 64)

        self.final = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(128, out_channels, 4, padding=1),
            nn.Tanh(),
        )

    def forward(self, x, semantic_vec):
        # U-Net generator with skip connections from encoder to decoder
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)

        semantic_vec = semantic_vec.unsqueeze(2)
        semantic_vec = semantic_vec.unsqueeze(3)
        d8 = torch.cat((d8, semantic_vec), 1)

        u1 = self.up1(d8, d7)
        u2 = self.up2(u1, d6)
        u3 = self.up3(u2, d5)
        u4 = self.up4(u3, d4)
        u5 = self.up5(u4, d3)
        u6 = self.up6(u5, d2)
        u7 = self.up7(u6, d1)

        return self.final(u7)

In [ ]:
#dataset.py

import glob
import random
import os
import numpy as np

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms


class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, mode="train"):
        self.transform = transforms.Compose(transforms_)

        self.files = sorted(glob.glob(os.path.join(root, mode) + "/*.*"))
        if mode == "train":
            self.files.extend(sorted(glob.glob(os.path.join(root, "test") + "/*.*")))

    def __getitem__(self, index):

        img = Image.open(self.files[index % len(self.files)])
        w, h = img.size
        img_A = img.crop((0, 0, w / 2, h))
        img_B = img.crop((w / 2, 0, w, h))
        img_A = self.transform(img_A)
        img_B = self.transform(img_B)

        return {"A": img_A, "B": img_B}

    def __len__(self):
        return len(self.files)


In [ ]:
#SSIM Function
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error

def ssim_function(real_img,fake_img,rand_img):#real image is a real test image, fake is reconstruction real_imgs and random image is random reconstructed image

  #sum_RI = 0
  #sum_RRI = 0
  img1 = img_as_float(real_img)
  img = color.rgb2gray(img1)
  rows, cols = img.shape

  #noise = np.ones_like(img) * 0.2 * (img.max() - img.min())
  #rng = np.random.default_rng()
  #noise[rng.random(size=noise.shape) > 0.5] *= -1

  img_noise = fake_img
  img_const = rand_img

  fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 4),
                          sharex=True, sharey=True)
  ax = axes.ravel()

  mse_none = mean_squared_error(img, img)
  ssim_none = ssim(img, img, data_range=img.max() - img.min())

  mse_noise = mean_squared_error(img, img_noise)
  ssim_noise = ssim(img, img_noise,
                    data_range=img_noise.max() - img_noise.min())

  mse_const = mean_squared_error(img, img_const)
  ssim_const = ssim(img, img_const,
                    data_range=img_const.max() - img_const.min())



  # ax[0].imshow(img, cmap=plt.cm.gray)
  ax[0].imshow(img)
  ax[0].set_xlabel(f'MSE: {mse_none:.2f}, SSIM: {ssim_none:.2f}')
  ax[0].set_title('real image')

  ax[1].imshow(img_noise)
  ax[1].set_xlabel(f'MSE: {mse_noise:.2f}, SSIM: {ssim_noise:.2f}')
  ax[1].set_title('reconstructed image')

  ax[2].imshow(img_const)
  ax[2].set_xlabel(f'MSE: {mse_const:.2f}, SSIM: {ssim_const:.2f}')
  ax[2].set_title('reconstructed random image')

  plt.tight_layout()
  plt.show()

  return ssim_noise, ssim_const

In [ ]:
class DecoderDataset(Dataset):
  def __init__(self, fmri_train, train_imgs):
    self.datax = torch.tensor(fmri_train, dtype = torch.float32)
    self.datay = torch.tensor(train_imgs, dtype = torch.float32)

  def __getitem__(self, item):
    itemx = self.datax[item]
    itemy = self.datay[item].flatten()
    return itemx, itemy

  def __len__(self):
    return self.datax.shape[0]

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, patch_imgs, sp_imgs_v1, sp_imgs_v2, sp_imgs_v3, pixel):
    self.pix = pixel
    self.patch = patch_imgs
    self.sp_v1 = sp_imgs_v1
    self.sp_v2 = sp_imgs_v2
    self.sp_v3 = sp_imgs_v3


  def __getitem__(self, item):

    item_v1 = self.sp_v1[item, self.pix[0], self.pix[1]]
    item_v2 = self.sp_v2[item, self.pix[0], self.pix[1]]
    item_v3 = self.sp_v3[item, self.pix[0], self.pix[1]]
    item_patch = self.patch[item, self.pix[0], self.pix[1]]

    datax = torch.tensor(np.array((item_v1, item_v2, item_v3)), dtype= torch.float32)
    datay = torch.tensor(item_patch , dtype = torch.float32)

    return datax, datay

  def __len__(self):

    return self.patch.shape[0]

In [ ]:
#linear layer
class Aggregation(nn.Module):
    def __init__(self):
        super(Aggregation, self).__init__()
        self.fc = nn.Linear(in_features=3, out_features=1, bias = False )

    def forward(self, x):
        out = self.fc(x)
        return out

In [ ]:
def unpoolingAvg(mat, f, ori_shape):
    '''Undo an average-pooling of a 2d or 3d array to a larger size

    Args:
        mat (ndarray): 2d or 3d array to unpool on the first 2 dimensions.
        f (int): pooling kernel size.
        ori_shape (tuple): original shape to unpool to.
    Returns:
        result (ndarray): <mat> unpoolled to shape <ori_shape>.
    '''
    m, n = ori_shape[:2]
    ny = m//f
    nx = n//f

    tmp = np.reshape(mat, (ny, 1, nx, 1)+mat.shape[2:])
    tmp = np.repeat(tmp, f, axis=1)
    tmp = np.repeat(tmp, f, axis=3)
    tmp = np.reshape(tmp, (ny*f, nx*f)+mat.shape[2:])
    result=np.zeros(ori_shape)

    result[:tmp.shape[0], :tmp.shape[1], ...]= tmp

    return result

In [ ]:
class AverageMeter(object):
  """Computes and stores the average and current value"""
  def __init__(self):
    self.reset()
  def reset(self):
    self.val = 0
    self.avg = 0
    self.sum = 0
    self.count = 0
  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

In [ ]:
#cuda
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
'''
Training contour decoders
Reconstruct shapes of stimulus images from the fMRI inputs
'''
import argparse
import os
import numpy as np
import math
import itertools
from PIL import Image
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
import torch.nn as nn
import torch
import torchvision
import matplotlib.pyplot as plt
import scipy
import random


np.random.seed(0)
def rasterize_img(img, out_size):
    raw_sz = img.shape[1]
    per_check = raw_sz/out_size
    new_arr = np.zeros((1, out_size, out_size))
    for l in range(out_size):
        for p in range(out_size):
            sx = int(l*per_check)
            ex = int((l+1)*per_check)
            sy = int(p*per_check)
            ey = int((p+1)*per_check)
            check = img[0, sx:ex, sy:ey]
            sum = np.sum(np.reshape(check, (check.size,)))
            avg = sum*1.0 / (per_check*per_check)
            new_arr[0, l, p] = avg
    return new_arr


def de_rasterize_img(img, out_size):
    raw_sz = img.shape[1]
    times = out_size/raw_sz
    new_arr = np.zeros((1, out_size, out_size))
    for l in range(raw_sz):
        for p in range(raw_sz):
            sx = int(l*times)
            ex = int((l+1)*times)
            sy = int(p*times)
            ey = int((p+1)*times)
            new_arr[0, sx:ex, sy:ey] = img[0, l, p]
    return new_arr


class Decoder_sh(nn.Module):
    def __init__(self, fmri_size, latent_dim):
        super(Decoder_sh, self).__init__()

        self.model = nn.Sequential(
            # nn.Linear(fmri_size, 1400),#1765
            # nn.Sigmoid(),
            # nn.Linear(1400,1200),
            # nn.Sigmoid(),
            # nn.Linear(1200,latent_dim),
            # nn.Tanh()
            #*****
            nn.Linear(fmri_size,latent_dim),
            nn.Tanh()
            # nn.Linear(fmri_size, fmri_size),#1765
            # # nn.Sigmoid(), #Sigmoid
            # nn.Linear(fmri_size, 1300),
            # nn.Tanh(),
            # nn.Linear(1300,latent_dim),
            # nn.Tanh()
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        x = self.model(img_flat)
        return x


def save_demo1_images(sp_imgs, raw_imgs):
    for i in range(6000, 6050):
        plt.figure()
        im = sp_imgs[i]
        br = (np.array(im) * 255).astype(np.uint8)
        im = Image.fromarray(br[i])
        im = im.convert('RGB')

        zr = (np.array(raw_imgs) * 255).astype(np.uint8)
        raw_img = Image.fromarray(zr[i])
        raw_img = raw_img.convert('RGB')
        target = Image.new('RGB', (256 * 2, 256))
        target.paste(raw_img, (0, 0, 256, 256))
        target.paste(im, (256, 0, 512, 256))
        target = target.resize((512, 256), Image.ANTIALIAS)
        target = target.convert('L')
        target = np.asarray(target)
        target = target[:, ::-1]

        ax = plt.subplot(1, 1, 1)
        plt.imshow(np.fliplr(target), cmap='hot')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        plt.savefig("/content/drive/MyDrive/thesis/results/digits/shape_decoding%d.png" % i)


def decoding_shapes(train_fmri, train_imgs, test_fmri, test_imgs, output_size, n_epochs, batch_size,lr, b1=0.5, b2=0.9):


    latent_dim = output_size*output_size
    # rand_id = np.random.randint(low=0, high=train_fmri.shape[0], size=train_fmri.shape[0]) #shuffle with replacement
    rand_id = random.sample(range(train_fmri.shape[0]), train_fmri.shape[0]) #shuffle without replacement
    train_fmri = train_fmri[rand_id]
    train_imgs = output_encoder
    test_imgs = output_encoder_test
    fmri = np.concatenate([train_fmri, test_fmri])
    train_imgs = train_imgs[rand_id]
    imgs = np.concatenate([train_imgs, test_imgs])

    # raw_imgs =  np.concatenate([patch_train_imgs, patch_test_imgs])
    total_blocks = fmri.shape[0] #1250
    fmri_size = fmri.shape[1]

    train_num = total_blocks - 50 #10 bood
    batch_num = train_num / batch_size #9

    print('Train blocks:'+str(train_num))
    print('batch num:' + str(batch_num))
    cuda = True if torch.cuda.is_available() else False


    # Use binary cross-entropy loss
    adversarial_loss = torch.nn.BCELoss()
    pixelwise_loss = torch.nn.L1Loss()
    #pixelwise_loss = torch.nn.MSELoss()
    # pixelwise_loss = torch.nn.BCELoss()
    # Initialize generator and discriminator
    decoder = Decoder_sh(fmri_size=fmri_size, latent_dim=latent_dim)

    if cuda:
        decoder.cuda()
        adversarial_loss.cuda()
        pixelwise_loss.cuda()


    optimizer_E = torch.optim.Adam(decoder.parameters(), lr=lr)
    scheduler_E = torch.optim.lr_scheduler.StepLR(optimizer_E, step_size=5, gamma=0.7, last_epoch=-1)#step size 5 bood

    Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

    # ----------
    #  Training
    # ----------
    loss_train = []
    validation_loss = []
    fmri = torch.from_numpy(fmri)
    fmri = fmri.type(Tensor)
    imgs = torch.from_numpy(imgs)
    imgs = imgs.type(Tensor)
    for epoch in range(n_epochs + 1):
        decoder.train()
        for i in range(0, int(batch_num)):
            fmri_data = fmri[i * batch_size:(i + 1) * batch_size]
            print(list(fmri_data.size()))
            real_imgs = imgs[i * batch_size:(i + 1) * batch_size]
            optimizer_E.zero_grad()
            latent_vector = decoder(fmri_data)
            print("latent vec size")
            print(list(latent_vector.size()))
            obj_vector = real_imgs.reshape(real_imgs.shape[0], -1)
            e_loss = pixelwise_loss(obj_vector, latent_vector)
            e_loss.backward()
            optimizer_E.step()
            #print(type(e_loss))
            loss_train.append(e_loss.item())


            print(
                "[Epoch %d/%d] [Batch %d] [E loss: %f]"
                % (epoch, n_epochs, i, e_loss.item()) #train_loss
            )
            decoder.eval()
            with torch.no_grad():
              y_v_pred = decoder(fmri[train_num:train_num + batch_size])
              obj_test_vector = test_imgs.reshape(test_imgs.shape[0], -1)
              #print(type(obj_test_vector))
              obj_test_vector = torch.tensor(obj_test_vector, dtype = torch.float32)
              obj_test_vector = obj_test_vector.to("cuda")
              v_loss = pixelwise_loss(obj_test_vector, y_v_pred)
              validation_loss.append(v_loss.item())
            print(v_loss.item()) #test loss

        if epoch % 50 == 0: #10 bood
          test_fmri_data = fmri[train_num:train_num + batch_size]
          print("*" * 200)
          print(test_fmri_data.shape)
          latent_v = decoder(test_fmri_data)
          latent_v = latent_v.view(batch_size, 1, output_size, output_size)

          tempv = latent_v.data
    decoder.eval()
    imgs = decoder(fmri)
    imgs = imgs.view(fmri.shape[0], output_size, output_size)
    imgs = imgs.data.cpu() * 255.0
    imgs = np.asarray(imgs)


    plt.figure()
    plt.title('Loss of Base Decoder(V3) on Training Data')
    plt.plot(loss_train)
    plt.xlabel('number of epoch * number of batch')
    plt.ylabel('L1 loss')

    # xpoints = [i for i in range(1200)]

    # plt.plot(ypoints)
    plt.figure()
    # plt.plot(final_l)
    plt.title('Loss of Base Decoder(V3) on Test Data')
    plt.plot(validation_loss)
    plt.xlabel('number of epoch * number of batch')
    plt.ylabel('L1 loss')

    # plot_train(loss_train,validation_loss,'pixelwise loss','number of epoch * number of batch','loss')
    #save_demo1_images(imgs, raw_imgs)
    return decoder, imgs, rand_id


In [ ]:
# rand_id = random.sample(range(train_fmri.shape[0]), train_fmri.shape[0])

In [ ]:
# train_fmri.shape

# **Convolutional AE**

In [ ]:
class Encoder(nn.Module):

    def __init__(self, encoded_space_dim,fc2_input_dim):
        super().__init__()

        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=2, padding = 1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(True),
            nn.Conv2d(32, 64, 3, stride=2, padding = 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(True),
            nn.Conv2d(64, 128, 3, stride=2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(True),
            nn.Conv2d(128, 256, 3, stride=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(True)
        )

        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
        ### Linear section
        self.encoder_lin = nn.Sequential(
            nn.Linear(3 * 3 * 32, 256),
            nn.ReLU(True),
            nn.Linear(256, encoded_space_dim)
        )

    def forward(self, x):
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        x = self.encoder_lin(x)
        return x
class Decoder(nn.Module):

    def __init__(self, encoded_space_dim,fc2_input_dim):
        super().__init__()
        self.decoder_lin = nn.Sequential(
            nn.Linear(encoded_space_dim, 128),
            nn.ReLU(True),
            nn.Linear(256, 3 * 3 * 32),
            nn.ReLU(True)
        )

        self.unflatten = nn.Unflatten(dim=1,
        unflattened_size=(32, 3, 3))

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, output_padding=0),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(128, 64, 3, stride=2, output_padding=0),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(64, 32, 3, stride=2, output_padding=0),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(32, 8, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.LeakyReLU(True)
            # nn.ConvTranspose2d(8, 1, 3, stride=2, padding=1, output_padding=1)
        )

    def forward(self, x):
        x = self.decoder_lin(x)
        x = self.unflatten(x)
        x = self.decoder_conv(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
encoded_space_dim = 1024

In [ ]:
encoder_cnn = nn.Sequential(
  nn.Conv2d(1, 8, 3, stride=2, padding = 1),
  nn.BatchNorm2d(8),
  nn.LeakyReLU(True),
  nn.Conv2d(8, 16, 3, stride=2, padding = 1),
  nn.BatchNorm2d(16),
  nn.LeakyReLU(True),
  nn.Conv2d(16, 32, 3, stride=2, padding = 1),
  nn.BatchNorm2d(32),
  nn.LeakyReLU(True),
  nn.Conv2d(32, 64, 3, stride=2, padding = 1),
  nn.BatchNorm2d(64),
  nn.LeakyReLU(True),
  nn.Conv2d(64, 128, 3, stride=2, padding = 1),
  nn.BatchNorm2d(128),
  nn.LeakyReLU(True),
  nn.Conv2d(128, 256, 3, stride=2, padding = 1),
  nn.BatchNorm2d(256),
  nn.LeakyReLU(True)
  # nn.Conv2d(512, 1024, 3, stride=2, padding = 1),
  # nn.BatchNorm2d(1024),
  # nn.LeakyReLU(True)
)
flatten = nn.Flatten(start_dim=1)

# linear = nn.Sequential(
#   nn.Linear(262144, 8192),
#   nn.ReLU(True),
#   nn.Linear(8192, encoded_space_dim)
# )

In [ ]:
one_imgs = torch.tensor(patch_train_imgs[0] , dtype = torch.float32)
one_imgs = one_imgs.unsqueeze(dim = 0)
one_imgs = one_imgs.unsqueeze(dim = 0)
print(one_imgs.shape)

In [ ]:
out_put_encoder = encoder_cnn(one_imgs)
print(out_put_encoder.shape)

In [ ]:
output_flatten = flatten(out_put_encoder)
print(output_flatten.shape)

In [ ]:
output_linear = linear(output_flatten)
print(output_linear.shape)

In [ ]:
#Decoder
# decoder_lin = nn.Sequential(
#   nn.Linear(encoded_space_dim, 8192),
#   nn.ReLU(True),
#   nn.Linear(8192, 262144),
#   nn.ReLU(True)
# )

unflatten = nn.Unflatten(dim=1,unflattened_size=(256, 2, 2))

decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1,output_padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1,output_padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(16, 8, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(8, 1, 3, stride=2, padding=1, output_padding=1)
            # nn.BatchNorm2d(1),
            # nn.LeakyReLU(True)
)

In [ ]:
output_linearD = decoder_lin(output_flatten)
print(output_linearD.shape)

In [ ]:
output_unflatten = unflatten(output_flatten)
print(output_unflatten.shape)

In [ ]:
out_put_decoder = decoder_conv(output_unflatten)
print(out_put_decoder.shape)

# AE

In [ ]:
#VAE(Variational Auto encoder)
class Encoder(nn.Module):
    def __init__(self, latent_dims):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(16384, 5000) #2^16
        self.linear2 = nn.Linear(5000, latent_dims)
        # self.linear3 = nn.Linear(2000, latent_dims)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        # x = F.relu(self.linear2(x))
        return self.linear2(x)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 5000)
        self.linear2 = nn.Linear(5000, 16384)
        # self.linear3 = nn.Linear(4000, 10000)

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        # z = torch.sigmoid(self.linear3(z))
        return z


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [ ]:
def train(autoencoder, data, epochs=150):
    opt = torch.optim.Adam(autoencoder.parameters(), lr=1e-4)
    loss_train = []
    for epoch in range(epochs):
        for x in data:
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum()
            loss.backward()
            opt.step()
            loss_train.append(loss.item())
    return autoencoder,loss_train

In [ ]:
latent_dims = 1024
autoencoder = Autoencoder(latent_dims).to(device) # GPU

data = torch.utils.data.DataLoader(obj_vector_train,batch_size=50,shuffle= False)

autoencoder, loss_train = train(autoencoder, data)


In [ ]:
print((loss_train))

In [ ]:
# xpoints = [i for i in range(1200)]
plt.figure()
ypoints = loss_train
plt.title('Loss of Auto Encoder on Training Data')
plt.plot(ypoints)
plt.xlabel('number of epoch * number of batch size')
plt.ylabel('Loss')
plt.plot(ypoints)

In [ ]:
ypoints

In [ ]:
np.savetxt('AE_loss.csv', ypoints,delimiter=",")

In [ ]:
output_encoder = np.zeros((6000,1024))
# loss_test = []
for i , x in enumerate(data):
  z = autoencoder.encoder(x.to(device))
  z = z.to('cpu').detach().numpy()
  output_encoder[i*50:(i+1)*50] = z


In [ ]:
np.reshape(output_encoder,(6000,32,32))

In [ ]:
#test
data_test = torch.utils.data.DataLoader(obj_vector_test,batch_size=50,shuffle= False)
output_encoder_test = np.zeros((50,1024))
for i , x in enumerate(data_test):
  z = autoencoder.encoder(x.to(device))
  z = z.to('cpu').detach().numpy()
  output_encoder_test[i*50:(i+1)*50] = z

# VAE(Variational Auto Encoder)

In [ ]:
class Decoder_vae(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder_vae, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 5000)
        self.linear2 = nn.Linear(5000, 16384)
        # self.linear3 = nn.Linear(4000, 10000)

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        # z = torch.sigmoid(self.linear3(z))
        return z


In [ ]:
class VariationalEncoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalEncoder, self).__init__()
        self.linear1 = nn.Linear(16384, 5000)
        self.linear2 = nn.Linear(5000, latent_dims)
        self.linear3 = nn.Linear(5000, latent_dims)

        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.cuda() # hack to get sampling on the GPU
        self.N.scale = self.N.scale.cuda()
        self.kl = 0

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        mu =  self.linear2(x)
        sigma = torch.exp(self.linear3(x))
        z = mu + sigma*self.N.sample(mu.shape)
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        return z

In [ ]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = VariationalEncoder(latent_dims)
        self.decoder = Decoder_vae(latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [ ]:
def train(autoencoder, data, epochs=800):
    opt = torch.optim.Adam(autoencoder.parameters(), lr=1e-4)

    loss_train = []
    for epoch in range(epochs):
        for x in data:
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum() + autoencoder.encoder.kl
            loss.backward()
            opt.step()
            loss_train.append(loss.item())
    return autoencoder, loss_train

In [ ]:
latent_dims = 1024
data = torch.utils.data.DataLoader(obj_vector_train,batch_size=200,shuffle= False)
autoencoder = VariationalAutoencoder(latent_dims).to(device) # GPU
autoencoder, loss = train(autoencoder, data)

In [ ]:
output_aencoder = np.zeros((6000,1024))
# loss_test = []
for i , x in enumerate(data):
  z = autoencoder.encoder(x.to(device))
  z = z.to('cpu').detach().numpy()
  output_aencoder[i*200:(i+1)*200] = z


In [ ]:
#test
data_test = torch.utils.data.DataLoader(obj_vector_test,batch_size=200,shuffle= False)
output_aencoder_test = np.zeros((50,1024))
for i , x in enumerate(data_test):
  z = autoencoder.encoder(x.to(device))
  z = z.to('cpu').detach().numpy()
  output_aencoder_test[i*200:(i+1)*200] = z

In [ ]:
np.reshape(output_aencoder,(6000,32,32))

In [ ]:
output_encoder = output_aencoder
output_encoder_test = output_aencoder_test

In [ ]:

ypoints = loss
# plt.plot(ypoints, color='green', linestyle='dashed', linewidth=2, markersize=12)
plt.figure()
plt.title('Loss of VAE on Training Data')
plt.plot(ypoints)
plt.xlabel('number of epoch * number of batch size')
plt.ylabel('Loss')
plt.plot(ypoints)

In [ ]:
loss

In [ ]:
np.savetxt('loss_VAE.csv',loss, delimiter = ",")

# **Conv AE for MY Prob**

In [ ]:
class CustomDataset1(Dataset):
  def __init__(self, patch_imgs):
    self.patch = torch.tensor(patch_imgs, dtype = torch.float32)

  def __getitem__(self, item):
    datax = self.patch[item,:,:]
    datax = torch.unsqueeze(datax, dim = 0)
    return datax

  def __len__(self):

    return self.patch.shape[0]

In [ ]:
train_set = CustomDataset1(patch_train_imgs)
test_set = CustomDataset1(patch_test_imgs)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=200, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=200, shuffle = False)

In [ ]:
x = next(iter(test_loader))
print(x.shape)

In [ ]:
encoded_space_dim = 1024

In [ ]:
class Encoder(nn.Module):

    def __init__(self, encoded_space_dim):
        super().__init__()

        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
          nn.Conv2d(1, 8, 3, stride=2, padding = 1),
          nn.BatchNorm2d(8),
          nn.LeakyReLU(True),
          nn.Conv2d(8, 16, 3, stride=2, padding = 1),
          nn.BatchNorm2d(16),
          nn.LeakyReLU(True),
          nn.Conv2d(16, 32, 3, stride=2, padding = 1),
          nn.BatchNorm2d(32),
          nn.LeakyReLU(True),
          nn.Conv2d(32, 64, 3, stride=2, padding = 1),
          nn.BatchNorm2d(64),
          nn.LeakyReLU(True),
          nn.Conv2d(64, 128, 3, stride=2, padding = 1),
          nn.BatchNorm2d(128),
          nn.LeakyReLU(True),
          nn.Conv2d(128, 256, 3, stride=2, padding = 1),
          nn.BatchNorm2d(256),
          nn.LeakyReLU(True)
        # nn.Conv2d(512, 1024, 3, stride=2, padding = 1),
        # nn.BatchNorm2d(1024),
        # nn.LeakyReLU(True)
        )

        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
        # ### Linear section
        # self.encoder_lin = nn.Sequential(
        #     nn.Linear(65536, 4096),
        #     nn.ReLU(True),
        #     nn.Linear(4096, 1024)
        # )

    def forward(self, x):
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        # x = self.encoder_lin(x)
        return x

class Decoder(nn.Module):

    def __init__(self, encoded_space_dim):
        super().__init__()
        # self.decoder_lin = nn.Sequential(
        #     nn.Linear(1024, 4096),
        #     nn.ReLU(True),
        #     nn.Linear(4096, 65536),
        #     nn.ReLU(True)
        # )

        self.unflatten = nn.Unflatten(dim=1,
        unflattened_size=(256, 2, 2))

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1,output_padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1,output_padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(16, 8, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.LeakyReLU(True),
            nn.ConvTranspose2d(8, 1, 3, stride=2, padding=1, output_padding=1)
        )

    def forward(self, x):
        # x = self.decoder_lin(x)
        x = self.unflatten(x)
        x = self.decoder_conv(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
### Define the loss function
loss_fn = torch.nn.MSELoss()

### Define an optimizer (both for the encoder and the decoder!)
lr= 0.0001

### Set the random seed for reproducible results
torch.manual_seed(0)

### Initialize the two networks
d = 1024

#model = Autoencoder(encoded_space_dim=encoded_space_dim)
encoder = Encoder(encoded_space_dim=d)
decoder = Decoder(encoded_space_dim=d)
params_to_optimize = [
    {'params': encoder.parameters()},
    {'params': decoder.parameters()}
]

optim = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=1e-5)

# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

# Move both the encoder and the decoder to the selected device
encoder.to(device)
decoder.to(device)

In [ ]:
### Training function
def train_epoch(encoder, decoder, device, dataloader, loss_fn, optimizer):
    # Set train mode for both the encoder and the decoder
    encoder.train()
    decoder.train()
    train_loss = []
    n = []
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for image_batch in dataloader: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        # Move tensor to the proper device
        image_batch = image_batch.to(device)
        # Encode data
        encoded_data = encoder(image_batch)
        # Decode data
        decoded_data = decoder(encoded_data)
        # Evaluate loss
        loss = loss_fn(decoded_data, image_batch)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        n.append(encoded_data)
        # Print batch loss
        print('\t partial train loss (single batch): %f' % (loss.data))
        train_loss.append(loss.detach().cpu().numpy())


    return np.mean(train_loss)

In [ ]:
num_epochs = 200
diz_loss = {'train_loss':[]}
final_l = []
for epoch in range(num_epochs):
   train_loss =train_epoch(encoder,decoder,device,train_loader,loss_fn,optim)
   final_l.append(train_loss)
   diz_loss['train_loss'].append(train_loss)


In [ ]:
a = np.array(final_l)

In [ ]:
# xpoints = [i for i in range(1200)]
ypoints = final_l
# plt.plot(ypoints)
plt.figure()
# plt.plot(final_l)
plt.title('Loss of CAE on Training Data')
plt.plot(ypoints)
plt.xlabel('number of epoch')
plt.ylabel('Loss')


In [ ]:
# xpoints = [i for i in range(1200)]
ypoints = final_l
plt.plot(ypoints)

In [ ]:
np.savetxt("final_l.csv", final_l, delimiter=",")

In [ ]:
output_encoder = np.zeros((6000,1024))
# loss_test = []
for i , x in enumerate(train_loader):
  z = encoder(x.to(device))
  z = z.to('cpu').detach().numpy()
  output_encoder[i*200:(i+1)*200] = z
  # print(z.shape)


In [ ]:
#test
output_encoder_test = np.zeros((50,1024))
for i , x in enumerate(test_loader):
  z = encoder(x.to(device))
  z = z.to('cpu').detach().numpy()
  output_encoder_test[i*200:(i+1)*200] = z

In [ ]:
print(output_encoder_test.shape)

# **8. Training shape decoder**

In [ ]:
#shape decoder model


print('Training shape decoder:')
#sp_decoder, sp_imgs, raw_imgs = decoding_shapes(train_lvc, train_imgs, test_lvc, test_imgs, output_size=32, n_epochs=101, batch_size=10, lr=0.0006)
sp_decoder_v1, sp_imgs_v1, rand_id1 = decoding_shapes(fmri_train_v1, output_encoder, fmri_test_v1, output_encoder_test, output_size=32, n_epochs=600, batch_size=50, lr=0.000005)
sp_decoder_v2, sp_imgs_v2, rand_id2 = decoding_shapes(fmri_train_v2, output_encoder, fmri_test_v2, output_encoder_test, output_size=32, n_epochs=600, batch_size=50, lr=0.000005)
sp_decoder_v3, sp_imgs_v3, rand_id3 = decoding_shapes(fmri_train_v3, output_encoder, fmri_test_v3, output_encoder_test, output_size=32, n_epochs=600, batch_size=50, lr=0.000005)

In [ ]:
# raw_imgs_v1 = np.concatenate([patch_train_imgs, patch_aug_imgs])
raw_imgs_v1 = np.concatenate([patch_train_imgs, patch_test_imgs])
newSize = 32
raw_imgs_v2 = np.zeros((6050,32,32))
for i in range(6050):
      raw_imgs_v2[i,:,:] = cv2.resize(raw_imgs_v1[i,:,:], dsize = (newSize, newSize), interpolation=cv2.INTER_CUBIC)

raw_imgs_v1 = raw_imgs_v2
raw_imgs_v3 = raw_imgs_v1

In [ ]:
for i in range(50):
  print(raw_imgs_v1[i].min())

**`Save Shape decoder model`**

In [ ]:
output_dict1 = {"sp_imgs_1":sp_imgs_v1, "rand_id1": rand_id1 }
#, "rand_id1": rand_id1
torch.save(output_dict1,"/content/drive/MyDrive/thesis/shape decoder model/output_dict1.pth")

In [ ]:
output_dict2 = {"sp_imgs_2":sp_imgs_v2, "rand_id2": rand_id2}
torch.save(output_dict2,"/content/drive/MyDrive/thesis/shape decoder model/output_dict2.pth")

In [ ]:
output_dict3 = {"sp_imgs_3":sp_imgs_v3, "rand_id3": rand_id3}
torch.save(output_dict3,"/content/drive/MyDrive/thesis/shape decoder model/output_dict3.pth")

In [ ]:
%cd /content/drive/MyDrive/thesis/shape decoder model
torch.save(sp_decoder_v1.state_dict(), 'sp_decoder_v1.pth')
torch.save(sp_decoder_v2.state_dict(), 'sp_decoder_v2.pth')
torch.save(sp_decoder_v3.state_dict(), 'sp_decoder_v3.pth')

**`Loading Shape decoder`**

In [ ]:
load_dict_1 = torch.load("/content/drive/MyDrive/thesis/shape decoder model/output_dict1.pth")
load_dict_2 = torch.load("/content/drive/MyDrive/thesis/shape decoder model/output_dict2.pth")
load_dict_3 = torch.load("/content/drive/MyDrive/thesis/shape decoder model/output_dict3.pth")

In [ ]:
sp_imgs_v1 = load_dict_1["sp_imgs_1"]
sp_imgs_v2 = load_dict_2["sp_imgs_2"]
sp_imgs_v3 = load_dict_3["sp_imgs_3"]

# raw_imgs_v1 = load_dict_1["raw_imgs_1"]
# raw_imgs_v2 = load_dict_2["raw_imgs_2"]
# raw_imgs_v3 = load_dict_3["raw_imgs_3"]

rand_id1 = load_dict_1["rand_id1"]
rand_id2 = load_dict_2["rand_id2"]
rand_id3 = load_dict_3["rand_id3"]

In [ ]:
fmri = np.concatenate([fmri_train_v1, fmri_test_v1])
fmri_size = fmri.shape[1]
latent_dim = 1024
sp_decoder_v1 = Decoder_sh(fmri_size,latent_dim)
sp_decoder_v1.to(device)
sp_decoder_v1.load_state_dict(torch.load('/content/drive/MyDrive/thesis/shape decoder model/sp_decoder_v1.pth'))

In [ ]:
fmri = np.concatenate([fmri_train_v2, fmri_test_v2])
fmri_size = fmri.shape[1]
latent_dim = 1024
sp_decoder_v2 = Decoder_sh(fmri_size,latent_dim)
sp_decoder_v2.to(device)
sp_decoder_v2.load_state_dict(torch.load('/content/drive/MyDrive/thesis/shape decoder model/sp_decoder_v2.pth'))

In [ ]:
fmri = np.concatenate([fmri_train_v3, fmri_test_v3])
fmri_size = fmri.shape[1]
latent_dim = 1024
sp_decoder_v3 = Decoder_sh(fmri_size,latent_dim)
sp_decoder_v3.to(device)
sp_decoder_v3.load_state_dict(torch.load('/content/drive/MyDrive/thesis/shape decoder model/sp_decoder_v3.pth'))

**`Plot Reconstructed Images`**

In [ ]:


data = raw_imgs_v1[1]
# data = np.transpose(data)
img = np.reshape(data, (32, 32))
matplotlib.pyplot.imshow(img, cmap = 'gray')
plt.show()

In [ ]:
for i in range(0,50):
  print('reconstructed image from V1')
  matplotlib.pyplot.imshow(patch_test_imgs[i], cmap = 'gray')
  plt.show()

In [ ]:
#plot images

# # matplotlib.pyplot.imshow(outPut_test[40,:,:])
# # plt.show()
for i in range(6000,6050):
  print('reconstructed image from V1')
  matplotlib.pyplot.imshow(raw_imgs_v1[i], cmap = 'gray')
  plt.show()
  # print('reconstructed image from V2')
  # matplotlib.pyplot.imshow(sp_imgs_v2[i], cmap = 'gray')
  # plt.show()
  # print('reconstructed image from V3')
  # matplotlib.pyplot.imshow(sp_imgs_v3[i], cmap = 'gray')
  # # plt.show()
  # print('raw image %d'%i)
  # matplotlib.pyplot.imshow(np.transpose(raw_imgs_v1[i]), cmap = 'gray')
  # plt.show()
  print('reconstructed image from V1 + V2 + v3')
  matplotlib.pyplot.imshow(1/3 * sp_imgs_v1[i]+ 1/3 * sp_imgs_v2[i]+ 1/3 * sp_imgs_v3[i], cmap = 'gray')
  plt.show()

# print("reconstructed image")

# matplotlib.pyplot.imshow(sp_imgs_v1[9])
# plt.show()
# print("real image")
# matplotlib.pyplot.imshow(raw_imgs_v1[9])
# plt.show()


In [ ]:
import matplotlib
matplotlib.pyplot.imshow(sp_imgs_v1[1])
plt.show()
matplotlib.pyplot.imshow(sp_imgs_v2[1])
plt.show()
matplotlib.pyplot.imshow(sp_imgs_v3[1])
plt.show()

In [ ]:
# ordering
import matplotlib
# new_raw_v1 = np.zeros(raw_imgs_v1.shape)
new_sp_v1 = np.zeros(sp_imgs_v1.shape)

for id,i in enumerate(rand_id1):
  # print(id)
  # print(i)
  # new[i] = t_fmri[id]
  # new_raw_v1[i] = raw_imgs_v1[id]
  new_sp_v1[i] = sp_imgs_v1[id]


# print(new_raw_v1.shape)
# matplotlib.pyplot.imshow(new_raw_v1[0])
# plt.show()

# new_raw_v2 = np.zeros(raw_imgs_v2.shape)
new_sp_v2 = np.zeros(sp_imgs_v2.shape)
for id,i in enumerate(rand_id2):
  # print(id)
  # print(i)
  # new[i] = t_fmri[id]
  # new_raw_v2[i] = raw_imgs_v2[id]
  new_sp_v2[i] = sp_imgs_v2[id]
# matplotlib.pyplot.imshow(new_raw_v2[0])
# plt.show()


# new_raw_v3 = np.zeros(raw_imgs_v3.shape)
new_sp_v3 = np.zeros(sp_imgs_v3.shape)
for id,i in enumerate(rand_id3):
  # print(id)
  # print(i)
  # new[i] = t_fmri[id]
  # new_raw_v3[i] = raw_imgs_v3[id]
  new_sp_v3[i] = sp_imgs_v3[id]

# matplotlib.pyplot.imshow(new_raw_v3[0])
# plt.show()


# new_raw_v1[6000:6050] = raw_imgs_v1[6000:6050]
# new_raw_v2[6000:6050] = raw_imgs_v2[6000:6050]
# new_raw_v3[6000:6050] = raw_imgs_v3[6000:6050]

new_sp_v1[6000:6050] = sp_imgs_v1[6000:6050]
new_sp_v2[6000:6050] = sp_imgs_v2[6000:6050]
new_sp_v3[6000:6050] = sp_imgs_v3[6000:6050]

# raw_imgs_v1 = new_raw_v1
# raw_imgs_v2 = new_raw_v2
# raw_imgs_v3 = new_raw_v3

sp_imgs_v1 = new_sp_v1
sp_imgs_v2 = new_sp_v2
sp_imgs_v3 = new_sp_v3

# **9. Linear Layer for Aggregation**

In [ ]:
raw_imgs_v2.shape

In [ ]:
newSize = 32
raw_imgs_v2 = np.zeros((6050,32,32))
for i in range(6050):
      raw_imgs_v2[i,:,:] = cv2.resize(raw_imgs_v1[i,:,:], dsize = (newSize, newSize), interpolation=cv2.INTER_CUBIC)

raw_imgs_v1 = raw_imgs_v2
raw_imgs_v3 = raw_imgs_v1

In [ ]:
print(raw_imgs_v3.shape)

In [ ]:
#Linear

#train model
import torch
torch.manual_seed(1)
t_loss = []
t1_loss =[]
for ix in range(10):
  for iy in range(32):
    train_set = CustomDataset(patch_imgs = raw_imgs_v1[0:6000], sp_imgs_v1 = sp_imgs_v1[0:6000,:,:],sp_imgs_v2 = sp_imgs_v2[0:6000,:,:], sp_imgs_v3 = sp_imgs_v3[0:6000,:,:], pixel = (ix,iy))
    trainloader = DataLoader(train_set, batch_size = 100, shuffle= False)
    result = Aggregation()
    b = torch.tensor([1/3 , 1/3 , 1/3])
    b = torch.unsqueeze(b , dim = 0)
    result.fc.weight = nn.Parameter(b)
    result.to(device)
    result_loss = nn.MSELoss()
    result_optimizer = torch.optim.Adam(params = result.parameters(), lr = 0.0001)

    for i in range(120):
      total_loss = AverageMeter()
      result.train()

      for x,y in trainloader:
        x = x.to(device)
        y = y.to(device)
        ypred = result(x)
        m_loss = result_loss(y,ypred)
        m_loss.backward()
        result_optimizer.step()
        result_optimizer.zero_grad()
        total_loss.update(m_loss.item())
        # t_loss.append(total_loss)
      #save Linear model

      print(f"pixel: ({ix, iy}), epoch:{i},loss: {total_loss.avg}")
      t_loss.append(total_loss.avg)
      t1_loss.append(t_loss)

      t_loss = []
    path = f"/content/drive/MyDrive/thesis/linear_model_new_decoder/Linear_{ix}_{iy}.pt"
    torch.save(result.state_dict(), path)
    print("-"*50)

In [ ]:
plt.figure()
# plt.plot(final_l)
plt.title('Loss of Base Decoder(V3) on Test Data')
plt.plot(t1_loss.avg)
plt.xlabel('number of epoch * number of batch')
plt.ylabel('L1 loss')

In [ ]:
#Linear
#train model
loss_t = []
temp = []
final_loss = []
for ix in range(10,20):
  for iy in range(32):

    train_set = CustomDataset(patch_imgs = raw_imgs_v1[0:6000], sp_imgs_v1 = sp_imgs_v1[0:6000,:,:],sp_imgs_v2 = sp_imgs_v2[0:6000,:,:], sp_imgs_v3 = sp_imgs_v3[0:6000,:,:], pixel = (ix,iy))
    trainloader = DataLoader(train_set, batch_size = 100, shuffle= False)
    result = Aggregation()
    b = torch.tensor([1/3 , 1/3 , 1/3])
    b = torch.unsqueeze(b , dim = 0)
    result.fc.weight = nn.Parameter(b)
    result.to(device)
    result_loss = nn.MSELoss()
    result_optimizer = torch.optim.Adam(params = result.parameters(), lr = 0.0001)

    for i in range(120):

      total_loss = AverageMeter()
      result.train()
      temp = []

      for x,y in trainloader:
        x = x.to(device)
        y = y.to(device)
        ypred = result(x)
        m_loss = result_loss(y,ypred)
        m_loss.backward()
        result_optimizer.step()
        result_optimizer.zero_grad()
        total_loss.update(m_loss.item())
        temp.append(m_loss.item())
      final_loss.append(np.average(temp))

      #save Linear model
      print(f"pixel: ({ix, iy}), epoch:{i},loss: {total_loss.avg}")

    path = f"/content/drive/MyDrive/thesis/linear_model_new_decoder/Linear_{ix}_{iy}.pt"
    torch.save(result.state_dict(), path)
    print("-"*50)
plt.plot(final_loss)
plt.show()

In [ ]:
#Linear

#train model


for ix in range(20,32):
  for iy in range(32):
    train_set = CustomDataset(patch_imgs = raw_imgs_v1[0:6000], sp_imgs_v1 = sp_imgs_v1[0:6000,:,:],sp_imgs_v2 = sp_imgs_v2[0:6000,:,:], sp_imgs_v3 = sp_imgs_v3[0:6000,:,:], pixel = (ix,iy))
    trainloader = DataLoader(train_set, batch_size = 100, shuffle= False)
    result = Aggregation()
    b = torch.tensor([1/3 , 1/3 , 1/3])
    b = torch.unsqueeze(b , dim = 0)
    result.fc.weight = nn.Parameter(b)
    result.to(device)
    result_loss = nn.MSELoss()
    result_optimizer = torch.optim.Adam(params = result.parameters(), lr = 0.0001)

    for i in range(120):
      total_loss = AverageMeter()
      result.train()

      for x,y in trainloader:
        x = x.to(device)
        y = y.to(device)
        ypred = result(x)
        m_loss = result_loss(y,ypred)
        m_loss.backward()
        result_optimizer.step()
        result_optimizer.zero_grad()
        total_loss.update(m_loss.item())
      #save Linear model

      print(f"pixel: ({ix, iy}), epoch:{i},loss: {total_loss.avg}")

    path = f"/content/drive/MyDrive/thesis/linear_model_new_decoder/Linear_{ix}_{iy}.pt"
    torch.save(result.state_dict(), path)
    print("-"*50)

In [ ]:
#load Linear Models for train Phase
outPut = np.zeros((6000,32,32))
#outPut = outPut.to(device)
for ix in range(32):
  for iy in range(32):
    train_set = CustomDataset(patch_imgs = raw_imgs_v1[0:6000], sp_imgs_v1 = sp_imgs_v1[0:6000,:,:],sp_imgs_v2 = sp_imgs_v2[0:6000,:,:], sp_imgs_v3 = sp_imgs_v3[0:6000,:,:], pixel = (ix,iy))
    trainloader = DataLoader(train_set, batch_size = 6000, shuffle= False)
    linear_model = Aggregation()
    path = f"/content/drive/MyDrive/thesis/linear_model_new_decoder/Linear_{ix}_{iy}.pt"
    linear_model.load_state_dict(torch.load(path))
    #linear_model.to(device)
    linear_model.eval()

    for x,y in trainloader:
      #x = x.to(device)
      #y = y.to(device)
      ypred = linear_model(x)

    #ypred.cpu()
    ypred_1 = ypred.squeeze()
    outPut[:,ix,iy] = ypred_1.detach().numpy()


In [ ]:
#Linear model for test Phase


outPut_test = np.zeros((50,32,32))
for ix in range(32):
  for iy in range(32):
    test_set = CustomDataset(patch_imgs = raw_imgs_v1[6000:6050], sp_imgs_v1 = sp_imgs_v1[6000:6050,:,:],sp_imgs_v2 = sp_imgs_v2[6000:6050,:,:], sp_imgs_v3 = sp_imgs_v3[6000:6050,:,:], pixel = (ix,iy))
    testloader = DataLoader(test_set, batch_size = 50, shuffle= False)
    linear_model = Aggregation()
    path = f"/content/drive/MyDrive/thesis/linear_model_new_decoder/Linear_{ix}_{iy}.pt"
    linear_model.load_state_dict(torch.load(path))
    #linear_model.to(device)
    linear_model.eval()

    for x,y in testloader:
      #x = x.to(device)
      #y = y.to(device)
      ypred = linear_model(x)

    #ypred.cpu()
    ypred_1 = ypred.squeeze()
    outPut_test[:,ix,iy] = ypred_1.detach().numpy()


In [ ]:
print('finish')

In [ ]:
#check weights of models
a = torch.load('/content/drive/MyDrive/thesis/linear_model_new_decoder/Linear_1_30.pt')
print(a)

In [ ]:
#check max value of pixels in images
import matplotlib
for i in range(6000):
  a = outPut[i,:,:].max()
  print(a)

In [ ]:
#plot images
import matplotlib
plt.show()
matplotlib.pyplot.imshow(outPut_test[0,:,:], cmap = 'gray')
#plt.show()
# matplotlib.pyplot.imshow(sp_imgs_v1[1200])
# plt.show()
# matplotlib.pyplot.imshow(sp_imgs_v2[1200])
plt.show()
matplotlib.pyplot.imshow(1/3 * sp_imgs_v1[6000]+ 1/3 * sp_imgs_v2[6000]+ 1/3 * sp_imgs_v3[6000], cmap = 'gray')
plt.show()
matplotlib.pyplot.imshow(raw_imgs_v1[6000], cmap = 'gray')


# **RGB array of Images**

**Change RGB Images and Build Array**

In [ ]:
#select Training images that correspond to stimulusID
import shutil
%cd /content/
# "convert tsv file into dictioary"
tsv_file = open("/content/drive/MyDrive/Stimulus_excel/stimulus_NaturalImageTraining.tsv")
read_tsv = csv. reader(tsv_file, delimiter="\t")
df = pd.DataFrame(read_tsv)
df = df.set_axis(['Name', 'ID','label','num'], axis=1, inplace=False)

df['label'] = df['label'].astype(float, errors = 'raise')
train_labels = df["label"]
#train_labels = np.array(train_labels)
print(train_labels.dtype)
df.drop(['label','num'] , inplace=True, axis=1)

dict_train = dict(zip(df.ID, df.Name))
print(dict_train)
nameStimulus = []
# "find ID's that correspond to Image name"
for ID in stimulusID:
  #res = {sub['name'] : sub['age'] for ID in test_list}
  res = {float(key) : dict_train[key] for key in dict_train}
  convK = float(ID)
  res = res[convK]
  nameStimulus.append(res)


print(nameStimulus)
#directory_path = '/content/drive/MyDrive/thesis/data/images/demo1/raw_imgs/'
original_path = '/content/drive/MyDrive/thesis/data/images/demo1/original_imgs/'
# original_path = '/content/drive/MyDrive/thesis/data/images/demo1/fff/'
target_path = '/content/drive/MyDrive/thesis/data/images/demo1/new_original/'

nameStimulus = nameStimulus[0:6000]
new_name = 0
for picture_name in nameStimulus:

  new_original = os.path.join(original_path, picture_name + ".JPEG")
  if os.path.isfile(new_original):
    new_target = os.path.join(target_path, str(new_name) + ".JPEG")
    shutil.copyfile(new_original, new_target)
    #new_dir = os.path.join(target_path, str(new_name) + ".JPEG")
    #os.rename(file_path, new_dir)
  new_original = ""
  new_target = ""
  new_name = int(new_name) + 1

In [ ]:
#select Test images that correspond to stimulusID
import shutil
%cd /content/
# "convert tsv file into dictioary"
test_file = open("/content/drive/MyDrive/Stimulus_excel/stimulus_NaturalImageTest.tsv")
test_tsv = csv. reader(test_file, delimiter="\t")
df_test = pd.DataFrame(test_tsv)
df_test = df_test.set_axis(['Name', 'ID','label','num'], axis=1, inplace=False)

df_test['label'] = df_test['label'].astype(float, errors = 'raise')
test_labels = df_test['label']
df_test.drop(['label','num'] , inplace=True, axis=1)


dict_test = dict(zip(df_test.ID, df_test.Name))
#print(dict_test)

nameStimulus = []
# "find ID's that correspond to Image name"
for ID in stimulusID_test:

  res = {float(key) : dict_test[key] for key in dict_test}
  convK = float(ID)
  res = res[convK]
  nameStimulus.append(res)

#directory_path = '/content/drive/MyDrive/thesis/data/images/demo1/raw_imgs/'
original_path = '/content/drive/MyDrive/thesis/data/images/demo1/original_imgs/'
# original_path = '/content/drive/MyDrive/thesis/data/images/demo1/fff/'
target_path = '/content/drive/MyDrive/thesis/data/images/demo1/new_original/'

new_name = 6000
nameStimulus = nameStimulus[0:50]
for picture_name in nameStimulus:

  new_original = os.path.join(original_path, picture_name + ".JPEG")
  if os.path.isfile(new_original):
    new_target = os.path.join(target_path, str(new_name) + ".JPEG")
    shutil.copyfile(new_original, new_target)
    #new_dir = os.path.join(target_path, str(new_name) + ".JPEG")
    #os.rename(file_path, new_dir)
    new_original = ""
    new_target = ""
  else:
    new_original = ""
    new_target = ""
  new_name = int(new_name) + 1

**LOAD RGB IMAGES**

In [ ]:
import skimage.measure
import cv2
from keras.preprocessing import image

newSize = 256
pnew = []
num = 0
path = '/content/drive/MyDrive/thesis/data/images/demo1/new_original/'

for i in range(6050):
  real_imgs_rgb = plt.imread(path +'%d.JPEG' %i)
  # print(real_imgs_rgb)
  if len(real_imgs_rgb.shape) == 2:
    real_imgs_rgb = cv2.cvtColor(real_imgs_rgb, cv2.COLOR_GRAY2RGB)
  pnew.append(cv2.resize(real_imgs_rgb, (0,0),fx = 0.512 , fy = 0.512))
  # print(pnew[i-1])
  # print(i)

In [ ]:
patch_imgs_rgb = np.array(pnew)

In [ ]:
A = []
for i in range(250):
  first = patch_imgs_rgb[i]
  first = np.transpose(first,(2,0,1))

  A.append(first)

In [ ]:
RGB_imgs_1 = np.array(A)

In [ ]:
RGB_imgs_1 = torch.tensor(RGB_imgs_1 , dtype = torch.float32)
RGB_imgs_1 = RGB_imgs_1.to(device)

Save RGB Images into array

In [ ]:
num = 0
%cd '/content/drive/MyDrive/thesis/rgb_images/'
for i  in range(1):
  patch_one = RGB_imgs_1[num:num+250]
  output_imgs_rgb = {"patch_%d"%i:patch_one}
  torch.save(output_imgs_rgb,"/content/drive/MyDrive/thesis/shape decoder model/output_imgs_rgb_%d.pth" %i)
  num = num +250

In [ ]:
patch_one = RGB_imgs_1[6000:6050]
output_imgs_rgb = {"patch_test":patch_one}
torch.save(output_imgs_rgb,"/content/drive/MyDrive/thesis/shape decoder model/output_imgs_rgb_test.pth")

Load RGB Array

In [ ]:
RGB_imgs = np.zeros((250,3,256,256))
a = 0
b = 250
for i in range(1):
  patch_imgs_rgb = torch.load("/content/drive/MyDrive/thesis/shape decoder model/output_imgs_rgb_%d.pth" %i)

  # print(type(patch_imgs_rgb["patch_0"]))
  first = patch_imgs_rgb["patch_%d"%i]
  first = first.cpu().data.numpy()
  print(first)
  # plt.imshow(first)
  # first_1 = np.transpose(first,(0,1,2,3))
  RGB_imgs[a:b, :, :, :] = first
  a = a + 250
  b = b + 250

In [ ]:
RGB_imgs = torch.tensor(RGB_imgs , dtype = torch.float32)

In [ ]:
RGB_imgs = RGB_imgs.to(device)

In [ ]:
print(RGB_imgs[0].min())

In [ ]:
real_b = RGB_imgs.cpu().data

LOAD Test RGB

In [ ]:
RGB_test = np.zeros((50,3,256,256))

patch_imgs_rgb = torch.load("/content/drive/MyDrive/thesis/shape decoder model/output_imgs_rgb_test.pth")

first = patch_imgs_rgb["patch_test"]
print(first)
first = np.transpose(first,(0,3,1,2))
RGB_test[0:50, :, :, :] = first

RGB_test = torch.tensor(RGB_test , dtype = torch.float32)
RGB_test = RGB_test.to(device)

# **10. Saving unpooling Images**

In [ ]:
newSize = 256
raw_imgs_v2 = np.zeros((6050,256,256,3))
for i in range(6050):
      raw_imgs_v2[i,:,:] = cv2.resize(raw_imgs_v1[i,:,:], dsize = (newSize, newSize), interpolation=cv2.INTER_CUBIC)

raw_imgs_v1 = raw_imgs_v2
raw_imgs_v3 = raw_imgs_v1

In [ ]:
raw_imgs_v1 = patch_imgs_rgb

In [ ]:
raw_imgs_v2 = raw_imgs_v1
raw_imgs_v3 = raw_imgs_v2

In [ ]:
#unpooling train data
#unpool_spimages = []
unpool_trainimages = []
unpool_outPut = []
for i in range(0,1000):
  a  = outPut[i]
  # b = raw_imgs_v2[i]
  #c = outPut[i,:,:]
  unpool_outPut.append(unpoolingAvg(a, 8, (256,256)))
  unpool_trainimages.append(patch_imgs_rgb[i])
  #unpool_outPut.append(unpoolingAvg(c, 8, (256,256)))

# matplotlib.pyplot.imshow(unpool_outPut[0])
# plt.show()
# #print(unpool_spimages[1])

# matplotlib.pyplot.imshow(unpool_trainimages[0])
# plt.show()

In [ ]:
#save shape decoder outputs

for i in range(0, 1000):

    im = unpool_outPut[i]
    im = (np.array(im) * 255)
    im = Image.fromarray(im)
    im = im.convert('RGB')
    # z = unpool_trainimages[i]
    z = (np.array(unpool_trainimages))
    raw_img = Image.fromarray(z[i])
    raw_img = raw_img.convert('RGB')
    target = Image.new('RGB', (256 * 2, 256))
    target.paste(raw_img, (0, 0, 256, 256))
    target.paste(im, (256, 0, 512, 256))
    target = target.resize((512, 256), Image.ANTIALIAS)
    # print(i)
    target.save('/content/drive/MyDrive/thesis/data/images/demo1/samples/train/%d.jpg' % (i))
    # matplotlib.pyplot.imshow(target)
    # plt.show()
#.astype(np.uint8)


In [ ]:
%reset_selective unpool_trainimages

In [ ]:
#unpooling train data
#unpool_spimages = []
unpool_trainimages = []
unpool_outPut = []
for i in range(1000,2000):
  a  = outPut[i]
  b = raw_imgs_v2[i]
  #c = outPut[i,:,:]
  unpool_outPut.append(unpoolingAvg(a, 8, (256,256)))
  # unpool_trainimages.append(unpoolingAvg(b, 8, (256,256)))
  unpool_trainimages.append(patch_imgs_rgb[i])
  #unpool_outPut.append(unpoolingAvg(c, 8, (256,256)))

# matplotlib.pyplot.imshow(unpool_outPut[0])
# plt.show()
# #print(unpool_spimages[1])

# matplotlib.pyplot.imshow(unpool_trainimages[0])
# plt.show()

In [ ]:
#save shape decoder outputs

for i in range(0, 1000):

    im = unpool_outPut[i]
    im = (np.array(im) * 255)
    im = Image.fromarray(im)
    im = im.convert('RGB')
    # z = unpool_trainimages[i]
    z = (np.array(unpool_trainimages))
    raw_img = Image.fromarray(z[i])
    raw_img = raw_img.convert('RGB')
    target = Image.new('RGB', (256 * 2, 256))
    target.paste(raw_img, (0, 0, 256, 256))
    target.paste(im, (256, 0, 512, 256))
    target = target.resize((512, 256), Image.ANTIALIAS)
    # print(i)
    target.save('/content/drive/MyDrive/thesis/data/images/demo1/samples/train/%d.jpg' % (i+1000))
    # matplotlib.pyplot.imshow(target)
    # plt.show()
#.astype(np.uint8)


In [ ]:
%reset_selective unpool_trainimages

In [ ]:
#unpooling train data
#unpool_spimages = []
unpool_trainimages = []
unpool_outPut = []
for i in range(2000,3000):
  a  = outPut[i]
  b = raw_imgs_v2[i]
  #c = outPut[i,:,:]
  unpool_outPut.append(unpoolingAvg(a, 8, (256,256)))
  # unpool_trainimages.append(unpoolingAvg(b, 8, (256,256)))
  unpool_trainimages.append(patch_imgs_rgb[i])
  #unpool_outPut.append(unpoolingAvg(c, 8, (256,256)))

# matplotlib.pyplot.imshow(unpool_outPut[0])
# plt.show()
# #print(unpool_spimages[1])

# matplotlib.pyplot.imshow(unpool_trainimages[0])
# plt.show()

In [ ]:
#save shape decoder outputs

for i in range(0, 1000):

    im = unpool_outPut[i]
    im = (np.array(im) * 255)
    im = Image.fromarray(im)
    im = im.convert('RGB')
    # z = unpool_trainimages[i]
    z = (np.array(unpool_trainimages))
    raw_img = Image.fromarray(z[i])
    raw_img = raw_img.convert('RGB')
    target = Image.new('RGB', (256 * 2, 256))
    target.paste(raw_img, (0, 0, 256, 256))
    target.paste(im, (256, 0, 512, 256))
    target = target.resize((512, 256), Image.ANTIALIAS)
    # print(i)
    target.save('/content/drive/MyDrive/thesis/data/images/demo1/samples/train/%d.jpg' % (i+2000))
    # matplotlib.pyplot.imshow(target)
    # plt.show()
#.astype(np.uint8)

In [ ]:
%reset_selective unpool_trainimages

In [ ]:
#unpooling train data
#unpool_spimages = []
unpool_trainimages = []
unpool_outPut = []
for i in range(3000,4000):
  a  = outPut[i]
  b = raw_imgs_v2[i]
  #c = outPut[i,:,:]
  unpool_outPut.append(unpoolingAvg(a, 8, (256,256)))
  # unpool_trainimages.append(unpoolingAvg(b, 8, (256,256)))
  unpool_trainimages.append(patch_imgs_rgb[i])
  #unpool_outPut.append(unpoolingAvg(c, 8, (256,256)))

# matplotlib.pyplot.imshow(unpool_outPut[0])
# plt.show()
# #print(unpool_spimages[1])

# matplotlib.pyplot.imshow(unpool_trainimages[0])
# plt.show()

In [ ]:
#save shape decoder outputs

for i in range(0, 1000):

    im = unpool_outPut[i]
    im = (np.array(im) * 255)
    im = Image.fromarray(im)
    im = im.convert('RGB')
    # z = unpool_trainimages[i]
    z = (np.array(unpool_trainimages))
    raw_img = Image.fromarray(z[i])
    raw_img = raw_img.convert('RGB')
    target = Image.new('RGB', (256 * 2, 256))
    target.paste(raw_img, (0, 0, 256, 256))
    target.paste(im, (256, 0, 512, 256))
    target = target.resize((512, 256), Image.ANTIALIAS)
    # print(i)
    target.save('/content/drive/MyDrive/thesis/data/images/demo1/samples/train/%d.jpg' % (i+3000))
    # matplotlib.pyplot.imshow(target)
    # plt.show()
#.astype(np.uint8)

In [ ]:
%reset_selective unpool_trainimages

In [ ]:
#unpooling train data
#unpool_spimages = []
unpool_trainimages = []
unpool_outPut = []
for i in range(4000,5000):
  a  = outPut[i]
  b = raw_imgs_v2[i]
  #c = outPut[i,:,:]
  unpool_outPut.append(unpoolingAvg(a, 8, (256,256)))
  # unpool_trainimages.append(unpoolingAvg(b, 8, (256,256)))
  unpool_trainimages.append(patch_imgs_rgb[i])
  #unpool_outPut.append(unpoolingAvg(c, 8, (256,256)))

# matplotlib.pyplot.imshow(unpool_outPut[0])
# plt.show()
# #print(unpool_spimages[1])

# matplotlib.pyplot.imshow(unpool_trainimages[0])
# plt.show()

In [ ]:
#save shape decoder outputs

for i in range(0, 1000):

    im = unpool_outPut[i]
    im = (np.array(im) * 255)
    im = Image.fromarray(im)
    im = im.convert('RGB')
    # z = unpool_trainimages[i]
    z = (np.array(unpool_trainimages))
    raw_img = Image.fromarray(z[i])
    raw_img = raw_img.convert('RGB')

    target = Image.new('RGB', (256 * 2, 256))
    target.paste(raw_img, (0, 0, 256, 256))
    target.paste(im, (256, 0, 512, 256))
    target = target.resize((512, 256), Image.ANTIALIAS)
    # print(i)
    target.save('/content/drive/MyDrive/thesis/data/images/demo1/samples/train/%d.jpg' % (i+4000))
    # matplotlib.pyplot.imshow(target)
    # plt.show()
#.astype(np.uint8)

In [ ]:
%reset_selective unpool_trainimages

In [ ]:
#unpooling train data
#unpool_spimages = []
unpool_trainimages = []
unpool_outPut = []
for i in range(5000,6000):
  a  = outPut[i]
  b = raw_imgs_v2[i]
  #c = outPut[i,:,:]
  unpool_outPut.append(unpoolingAvg(a, 8, (256,256)))
  # unpool_trainimages.append(unpoolingAvg(b, 8, (256,256)))
  unpool_trainimages.append(patch_imgs_rgb[i])
  #unpool_outPut.append(unpoolingAvg(c, 8, (256,256)))

# matplotlib.pyplot.imshow(unpool_outPut[0])
# plt.show()
# #print(unpool_spimages[1])

# matplotlib.pyplot.imshow(unpool_trainimages[0])
# plt.show()

In [ ]:
#save shape decoder outputs

for i in range(0, 1000):

    im = unpool_outPut[i]
    im = (np.array(im) * 255)
    im = Image.fromarray(im)
    im = im.convert('RGB')
    # z = unpool_trainimages[i]
    z = (np.array(unpool_trainimages))
    raw_img = Image.fromarray(z[i])
    raw_img = raw_img.convert('RGB')
    target = Image.new('RGB', (256 * 2, 256))
    target.paste(raw_img, (0, 0, 256, 256))
    target.paste(im, (256, 0, 512, 256))
    target = target.resize((512, 256), Image.ANTIALIAS)
    # print(i)
    target.save('/content/drive/MyDrive/thesis/data/images/demo1/samples/train/%d.jpg' % (i+5000))
    # matplotlib.pyplot.imshow(target)
    # plt.show()
#.astype(np.uint8)

In [ ]:
%reset_selective unpool_trainimages

In [ ]:
#unpooling test data
unpool_testimages = []
unpool_outPut_test = []
for i in range(0,50):
  b = raw_imgs_v2[i+6000]
  c = outPut_test[i,:,:]
  # unpool_testimages.append(unpoolingAvg(b, 8, (256,256)))
  unpool_testimages.append(patch_imgs_rgb[i+6000])
  unpool_outPut_test.append(unpoolingAvg(c, 8, (256,256)))

In [ ]:
for i in range(6000, 6050):

    im = unpool_outPut_test[i-6000]
    im = (np.array(im) * 255)
    im = Image.fromarray(im)
    im = im.convert('RGB')
    z = (np.array(unpool_testimages))
    raw_img = Image.fromarray(z[i-6000])
    raw_img = raw_img.convert('RGB')
    #display(raw_img)
    target = Image.new('RGB', (256 * 2, 256))
    target.paste(raw_img, (0, 0, 256, 256))
    target.paste(im, (256, 0, 512, 256))
    target = target.resize((512, 256), Image.ANTIALIAS)
    target.save('/content/drive/MyDrive/thesis/data/images/demo1/samples/val/%d.jpg' % i)

In [ ]:
print("done tonight")

In [ ]:
#unpooling Aug data
unpool_augimages = []
unpool_outPut_test = []
for i in range(0,1200):
  b = raw_imgs_v2[i+6000]
  # c = outPut_test[i,:,:]
  unpool_augimages.append(unpoolingAvg(b, 8, (256,256)))
  # unpool_outPut_test.append(unpoolingAvg(c, 8, (256,256)))

In [ ]:
for i in range(6000, 7200):

    # im = unpool_outPut_test[i-6000]
    # im = (np.array(im) * 255)
    # im = Image.fromarray(im)
    # im = im.convert('RGB')
    z = (np.array(unpool_augimages) * 255)
    raw_img = Image.fromarray(z[i-6000])
    raw_img = raw_img.convert('RGB')
    target = Image.new('RGB', (256 * 2, 256))
    target.paste(raw_img, (0, 0, 256, 256))
    target.paste(raw_img, (256, 0, 512, 256))
    target = target.resize((512, 256), Image.ANTIALIAS)
    target.save('/content/drive/MyDrive/thesis/data/images/demo1/samples/train/%d.jpg' % i)

# **12.final shape decoder outputs without Linear Layer**

In [ ]:
outPut = np.zeros((6000,32,32))
for i in range(6000):
  outPut[i,:,:] = 1/3 * sp_imgs_v1[i]+ 1/3 * sp_imgs_v2[i]+ 1/3 * sp_imgs_v3[i]

In [ ]:
outPut_test = np.zeros((50,32,32))
for i in range(6000,6050):
  outPut_test[i-6000,:,:] = 1/3 * sp_imgs_v1[i]+ 1/3 * sp_imgs_v2[i]+ 1/3 * sp_imgs_v3[i]

# **13. Training GAN**

In [ ]:
n_epochs = 15 #ghablan 10 bood
batch_size = 50
lr = 0.0002
b1 = 0.9
b2 = 0.999
img_height = 256
img_width = 256
channels = 3
dataset_name = '69digits'
interval = 2
test_sample_num = 50

cuda = True if torch.cuda.is_available() else False

# Loss functions
criterion_GAN = torch.nn.MSELoss()
criterion_pixelwise = torch.nn.MSELoss()
criteirion_contour = torch.nn.MSELoss()
# Loss weight of L1 pixel-wise loss between translated image and real image
lambda_pixel = 150

# Calculate output of image discriminator (PatchGAN)
patch = (3, img_height // 2 **4 , img_width // 2 ** 4)


# Initialize generator and discriminator
generator = GeneratorUNet()
discriminator = Discriminator()


if cuda:
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    criterion_GAN.cuda()
    criterion_pixelwise.cuda()
    criteirion_contour.cuda()


# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr = lr, betas=(b1, b2))
optimizer_D = torch.optim.SGD(discriminator.parameters(),  lr = lr)
scheduler_G = torch.optim.lr_scheduler.StepLR(optimizer_G, step_size=3, gamma=0.9)
scheduler_D = torch.optim.lr_scheduler.StepLR(optimizer_D, step_size=3, gamma=0.9)


# Configure dataloaders
transforms_ = [
    transforms.Resize((img_height, img_width), Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

# training set
dataloader = DataLoader(
    ImageDataset("/content/drive/MyDrive/thesis/data/images/demo1/samples/", transforms_=transforms_, mode='train'),
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
)

# validation set
val_dataloader = DataLoader(
    ImageDataset("/content/drive/MyDrive/thesis/data/images/demo1/samples/", transforms_=transforms_, mode="val"),
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
)



# Tensor type
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

print('Load semantic features:')
semantic_vecs = torch.load('/content/drive/MyDrive/thesis/data/sm_features/demo1/semantics')
# aug_train_new = torch.Tensor(aug_train)
train_semantic_vec = semantic_vecs[0:6000]
# train_semantic_vec = aug_train_new
test_semantic_vec = semantic_vecs[6000:6050]
train_semantic_vec = train_semantic_vec.type(Tensor)
test_semantic_vec = test_semantic_vec.type(Tensor)
prev_time = time.time()

generator_loss = []
discriminator_loss = []
GAN_loss = []
epoch_array = []
batch = []
k = 0
for epoch in range(0, n_epochs):
    scheduler_D.step()
    scheduler_G.step()
    for i, batch in enumerate(dataloader):
        # Model inputs
        real_A = batch["B"].type(Tensor)   # 轮廓 contours
        real_B = batch["A"].type(Tensor)   # 真图 real images
        # if k<= 5950:
          # real_B = RGB_imgs_1[k:k+50]
        # patch_imgs_rgb
        # Adversarial ground truths
        #normalization real A
        # a = real_A.cpu().detach().numpy()
        # real_A = cv2.normalize(a, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
        # b = a[:,:,0]
        # real_A = torch.from_numpy(b).float().to(device)
        #norm real B
        # a_b = real_B.cpu().detach().numpy()
        # a_b = a_b[0,:,:]
        # real_B = cv2.normalize(a_b, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
        # real_B = torch.from_numpy(real_B).float().to(device)
        valid = Tensor(np.ones((real_B.size(0), *patch)))
        fake = Tensor(np.zeros((real_B.size(0), *patch)))
        #Train Generator
        optimizer_G.zero_grad()

        # GAN loss
        fake_B = generator(real_A, train_semantic_vec[i*batch_size:(i+1)*batch_size])
        print(train_semantic_vec.shape)
        out_gen = fake_B
        # fake_B = generator(real_A)
        # test_a = fake_B
        pred_fake = discriminator(fake_B, real_A)
        loss_GAN = criterion_GAN(pred_fake, valid)
        # # Pixel-wise loss
        loss_pixel = criterion_pixelwise(fake_B, real_B)
        # Total loss
        loss_G = loss_GAN + lambda_pixel * loss_pixel
        loss_G.backward()
        optimizer_G.step()
        #save images into folder
        if i == 10:
          # for l in range(50):
          X_test = fake_B[5]
          a = X_test.cpu().detach().numpy()
          x_test = np.array(np.transpose(a, (1,2,0)))
          image_norm = cv2.normalize(x_test, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
          plt.imshow((image_norm*255).astype(np.uint8))
          plt.savefig('/content/drive/MyDrive/thesis/test-gen/reconstructed'+ str(epoch) + '_' + str(5))
            # X_train = real_B[l]
            # a_train = X_train.cpu().detach().numpy()
            # x_train = np.array(np.transpose(a_train, (1,2,0)))
            # image_norm_train = cv2.normalize(x_train, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
            # plt.imshow((image_norm_train*255).astype(np.uint8))
            # plt.savefig('/content/drive/MyDrive/thesis/gen_out_200/train_imgs'+ str(epoch) + '_' + str(l))


        #  Train Discriminator

        optimizer_D.zero_grad()

        # Real loss
        pred_real = discriminator(real_B, real_A)
        loss_real = criterion_GAN(pred_real, valid)

        # Fake loss
        pred_fake = discriminator(fake_B.detach(), real_A)
        print(pred_fake.shape)
        loss_fake = criterion_GAN(pred_fake, fake)

        # Total loss
        loss_D = 0.5 * (loss_real + loss_fake)

        loss_D.backward()
        optimizer_D.step()

        #  Log Progress
        batches_done = epoch * len(dataloader) + i
        batches_left = n_epochs * len(dataloader) - batches_done
        time_left = datetime.timedelta(seconds = batches_left * (time.time() - prev_time))
        prev_time = time.time()
        #Plot

        generator_loss.append(loss_G.item())
        discriminator_loss.append(loss_D.item())
        GAN_loss.append(loss_GAN.item())
        epoch_array.append(epoch)

        k =  k + 1
        print(
            "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, pixel: %f, adv: %f] ETA: %s"
            % (
                epoch,
                n_epochs,
                i,
                len(dataloader),
                loss_D.item(),
                loss_G.item(),
                loss_pixel.item(),
                loss_GAN.item(),
                time_left,
            )
        )
#plot
# Importing libraries
import matplotlib.pyplot as plt
import numpy as np
import math

# Using Numpy to create an array X
X = [i for i in range(len(dataloader)*n_epochs)]

# Assign variables to the y axis part of the curve
y = generator_loss
z = discriminator_loss

# Plotting both the curves simultaneously
plt.plot(X, y, color='r', label='Gan loss')
plt.figure()
plt.plot(X, z, color='g', label='discriminator loss')
# To load the display window
plt.show()

In [ ]:
criterion_GAN = torch.nn.MSELoss()
# Calculate output of image discriminator (PatchGAN)
patch = (1, img_height // 2 **4 , img_width // 2 ** 4)
valid = Tensor(np.ones((real_B.size(0), *patch)))
fake = Tensor(np.zeros((real_B.size(0), *patch)))
# Real loss
pred_real = discriminator(real_B, real_A)
loss_real = criterion_GAN(pred_real, valid)

# Fake loss
pred_fake = discriminator(fake_B.detach(), real_A)
loss_fake = criterion_GAN(pred_fake, fake)


**Save Generator **

In [ ]:
output_dict5 = {"fake_B":fake_B}
torch.save(output_dict5,"/content/drive/MyDrive/thesis/shape decoder model/generator.pth")

In [ ]:
a = real_A.cpu().detach().numpy()
a10 = np.array(np.transpose(a[1], (1,2,0)))
# plt.imshow(np.array(a10*255).astype(np.uint8))
print(a10[:,:,2])

In [ ]:
generator_one = torch.load("/content/drive/MyDrive/thesis/shape decoder model/generator.pth")
fake_B = generator_one["fake_B"]

In [ ]:
generator = GeneratorUNet()
discriminator = Discriminator()
cuda = True if torch.cuda.is_available() else False

if cuda:
    generator = generator.cuda()
    # discriminator = discriminator.cuda()
    # criterion_GAN.cuda()
    # criterion_pixelwise.cuda()
    # criteirion_contour.cuda()

In [ ]:
n_epochs = 200 #ghablan 10 bood
batch_size = 50
lr = 0.0002
b1 = 0.9
b2 = 0.999
img_height = 256
img_width = 256
channels = 3
dataset_name = '69digits'
interval = 2
test_sample_num = 50
transforms_ = [
    transforms.Resize((img_height, img_width), Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]
val_dataloader = DataLoader(
    ImageDataset("/content/drive/MyDrive/thesis/data/images/demo1/samples/", transforms_=transforms_, mode="val"),
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
)


In [ ]:
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
semantic_vecs = torch.load('/content/drive/MyDrive/thesis/data/sm_features/demo1/semantics')
# aug_train_new = torch.Tensor(aug_train)
train_semantic_vec = semantic_vecs[0:6000]
# train_semantic_vec = aug_train_new
test_semantic_vec = semantic_vecs[6000:6050]
train_semantic_vec = train_semantic_vec.type(Tensor)
test_semantic_vec = test_semantic_vec.type(Tensor)
prev_time = time.time()


In [ ]:
#@title
from SSIM_PIL import compare_ssim
from PIL import Image
from sklearn.metrics import jaccard_score
k = 0
s = 0
e = 50
fakex = []
realx = []
i = 0
SSIM_rec = np.array([])
SSIM_Random = np.array([])
value_list = np.array([])
similarity_array = np.array([])
value_list_random = np.array([])
# my_rho = np.corrcoef(x_simple, y_simple)************************************************ COrrelation Coefficient
h = 0
# for i in range(0,5):
for j, batch in enumerate(val_dataloader):
  if j == 0:
      plt.figure()
      real_A = batch["B"].type(Tensor)#contours
      real_B = batch["A"].type(Tensor)#real images
      fake_B = generator(real_A, test_semantic_vec[h:h+50]) #reconstructed images
      fake_B = fake_B.cpu().data
      fake_B = np.asarray(fake_B)
      real_test_one = fake_B
      real_B = real_B.cpu().data
      # real_b = RGB_test.cpu().data
      # real_B = np.asarray(real_B)
      fakex = fake_B
      realx = real_B
      h = h + 50
      if h >= 100:
        break
      for i in range(50):
          v = i+h-50
          print('**********%d*********'%v)
        #reconstructed image
          fimg = fake_B[i]
          fimg = np.array(np.transpose(fimg, (1,2,0)))
          fimg = cv2.normalize(fimg, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
          fimg = (np.array(fimg*255)).astype(np.uint8)
          # fimg = Image.fromarray(fimg)
        #real image
          rimg = realx[i]
          rimg = np.array(np.transpose(rimg, (1,2,0)))
          rimg = cv2.normalize(rimg, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
          rimg = (np.array(rimg*255)).astype(np.uint8)
          # rimg = Image.fromarray(rimg)
        #random reconstructed image
          m = random.randint(0,49)
          while m == i :
            m = random.randint(0,49)
          rand_fake = fake_B[m]
          rand_fake = np.array(np.transpose(rand_fake, (1,2,0)))
          rand_fake = cv2.normalize(rand_fake, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
          rand_fake = (np.array(rand_fake*255)).astype(np.uint8)
          # rand_fake = Image.fromarray(rand_fake)
      #save reconstructed images
          fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 4), sharex=True, sharey=True)
          ax = axes.ravel()

          ax[0].imshow(rimg)
          ax[0].set_title('real image')

          ax[1].imshow(fimg)
          ax[1].set_title('reconstructed image')

          ax[2].imshow(rand_fake)
          ax[2].set_title('reconstructed random image')

          plt.tight_layout()
          plt.show()

      plt.tight_layout()
      plt.savefig("/content/drive/MyDrive/thesis/results/digits/reconstructed_img%d.png"%s)

In [ ]:
# plt.imshow(real_A[1])
# realb = real_test_one.cpu().detach().numpy()
realb_new = np.array(np.transpose(real_test_one[20], (1,2,0)))
realb_new = cv2.normalize(realb_new, None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)
# plt.imshow((realb_new*255).astype(np.uint8) , cmap = "gray")
patch_imgs_rgb[6020] = cv2.normalize(patch_imgs_rgb[6020], None, alpha=0,beta=1, norm_type=cv2.NORM_MINMAX)


In [ ]:
#@title
from SSIM_PIL import compare_ssim
from PIL import Image
from sklearn.metrics import jaccard_score
k = 0
s = 0
e = 50
fakex = []
realx = []
i = 0
SSIM_rec = np.array([])
SSIM_Random = np.array([])
value_list = np.array([])
similarity_array = np.array([])
value_list_random = np.array([])
# my_rho = np.corrcoef(x_simple, y_simple)************************************************ COrrelation Coefficient
h = 0
# for i in range(0,5):
for j, batch in enumerate(val_dataloader):
  if j == 0:
      plt.figure()
      real_A = batch["B"].type(Tensor)#contours
      real_B = batch["A"].type(Tensor)#real images
      fake_B = generator(real_A, test_semantic_vec[h:h+50]) #reconstructed images
      fake_B = fake_B.cpu().data
      fake_B = np.asarray(fake_B)
      real_test_one = fake_B
      real_B = real_B.cpu().data
      # real_b = RGB_test.cpu().data
      # real_B = np.asarray(real_B)
      fakex = fake_B
      realx = real_B
      h = h + 50
      if h >= 100:
        break
      for i in range(50):
          v = i+h-50
          print('**********%d*********'%v)

        #reconstructed image
          # fimg = fakex[i]
          fimg = fake_B[i]
          fimg = 0.33 * fimg[0]+ 0.33* fimg[1]+ 0.33*fimg[2]
          # fimg[fimg < 0] = 0
          # fimg = fimg[:, ::-1]
          fimg = fimg.reshape(256, 256) * 255

        #real image
          rimg = realx[i]
          # rimg = RGB_test[i]
          rimg = 0.3 * rimg[0]+0.59 * rimg[1]+0.11*rimg[2]
          rimg[rimg < 0] = 0
          # rimg = rimg[:, ::-1]
          rimg = rimg.reshape(256, 256) * 255

        #random reconstructed image
          m = random.randint(0,49)
          while m == i :
            m = random.randint(0,49)
          # print(fimg.shape)
          rand_fake = fake_B[m]
          rand_fake = 0.3 * rand_fake[0]+0.59 * rand_fake[1]+0.11*rand_fake[2]
          rand_fake[rand_fake < 0] = 0
          # rand_fake = rand_fake[:, ::-1]
          rand_fake = rand_fake.reshape(256, 256) * 255

          # for k in range(256):
          #   for j in range(256):
          #     if rimg[k][j] <=0:
          #       rimg[k][j] = 1
          #     else:
          #       rimg[k][j] = 0

          # for k in range(256):
          #   for j in range(256):
          #     if rimg[k][j] <= (rimg.max())/2:
          #       rimg[k][j] = 0
          #     else:
          #       rimg[k][j] = 1

        # #binarization reconstructed image
        #   for k in range(256):
        #     for j in range(256):
        #       if fimg[k][j] <= (fimg.max())/2:
        #         fimg[k][j] = 0
        #       else:
        #         fimg[k][j] = 1

        # #binarization Random reconstructed image
        #   for k in range(256):
        #     for j in range(256):
        #       if rand_fake[k][j] <=  (rand_fake.max())/2:
        #         rand_fake[k][j] = 0
        #       else:
        #         rand_fake[k][j] = 1

          im = fimg
          im = (np.array(im)).astype(np.uint8)
          im = Image.fromarray(im)
          im = im.convert('RGB')
          real_test = im
          # im = cv2.multiply(im * patch_imgs_rgb[6000+i])#**********************

          rimg = (np.array(rimg)).astype(np.uint8)
          raw_img = Image.fromarray(rimg)
          raw_img = raw_img.convert('RGB')
          raw_imgs = raw_img
          randimg = rand_fake
          randimg = (np.array(randimg)).astype(np.uint8)
          randimg = Image.fromarray(randimg)
          randimg = randimg.convert('RGB')


          target = Image.new('RGB', (256 * 2, 256))
          target.paste(raw_img, (0, 0, 256, 256))
          target.paste(im, (256, 0, 512, 256))
          target = target.resize((512, 256), Image.ANTIALIAS)
          target = target.convert('L')
          target = np.asarray(target)

      #save reconstructed images
          # print(rimg.min())
          # print(rimg.max())
          # print(fimg.min())
          # print(fimg.max())
          im.save('/content/drive/MyDrive/thesis/reconstructed/%d.jpg' % i)
          ssim_noise, ssim_cons = ssim_function(rimg,fimg, rand_fake)
          SSIM_rec = np.append(SSIM_rec,ssim_noise)
          SSIM_Random = np.append(SSIM_Random,ssim_cons)

          # similarity = jaccard_score(rimg, fimg, average="micro")
          # similarity_array = np.append(similarity_array, similarity)
          print("new Value for ssim")
          value = compare_ssim(raw_img, im)
          value_list = np.append(value_list, value)
          print(value)
          # value_random = compare_ssim(raw_img, randimg)
          # value_list_random = np.append(value_list_random, value_random)
          # print(value_random)
          ax = plt.subplot(5, 10, i+1)
          plt.imshow(np.fliplr(target), cmap='hot')
          ax.get_xaxis().set_visible(False)
          ax.get_yaxis().set_visible(False)

      plt.tight_layout()
      plt.savefig("/content/drive/MyDrive/thesis/results/digits/reconstructed_img%d.png"%s)

In [ ]:
print(sum(value_list/50))

In [ ]:
print(sum(value_list_random)/50)

# **clipping**

In [ ]:
realb_new = np.array(np.transpose(real_test_one[20], (1,2,0)))
print(realb_new.min())

In [ ]:
multi_a = patch_imgs_rgb[6020] * realb_new

In [ ]:
plt.imshow(multi_a)

In [ ]:
plt.imshow(np.transpose(real_test_one[1], (1,2,0)))